# Imports

In [2]:
#Basic Imports
import json
import pandas as pd
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', None) 
import polars as pl
import numpy as np

# for loading keys securely and for saves
from dotenv import load_dotenv
import os
from pathlib import Path

# Yahoo fantasy things
import yfpy
from yfpy.query import YahooFantasySportsQuery
from yfpy import Data

# NFY Py Data things
import nflreadpy as nfl

# for API delays
import time

# Connect to Fantasy API

Load ENV file with API key obtained from Yahoo. ENV will also save tokens for continuous use rather than prividing a new token with every query

In [3]:
# Load the .env file
load_dotenv()

# Set target directory path for use of YFPY json load save
data_dir = Path('.') / "output"

# create YFPY Data instance for saving/loading data
data = Data(data_dir)

## Load yahoo base query

In [4]:
yahoo_query = YahooFantasySportsQuery(
   league_id='254906', #each season even if for the same league has a different league ID, this is for 2025
   game_code="nfl", # for nfl games - Yahoo has other sports
   game_id=461, #each season has a unique game ID, tyhis is for 2025
   yahoo_consumer_key=os.getenv('CONS_KEY'), #pull the API key from .env
   yahoo_consumer_secret=os.getenv('CONS_SECRET'), #pull the API secret from .env
   env_file_location=Path('.'), #location of .env
   env_var_fallback=True, #to fall back on env file for missing authentication info
   save_token_data_to_env_file=True #saves authentication token to env for continuous use rather than single use token)
)

## Prepare data for batch pulls

- Since API requests need season specific numbers and league IDs, create a multi level dictionary for automating API requests

In [5]:
# create dictionary with IDs found in YFPY documentation
seasonal_ids = {2025:{'season':461, 'l_id':'254906'},
                2024:{'season':449, 'l_id':'17240'},
                2023:{'season':423, 'l_id':'541098'},
                2022:{'season':414, 'l_id':'21504'},
                2021:{'season':406, 'l_id':'8243'},
                2020:{'season':399, 'l_id':'47784'},
                2019:{'season':390, 'l_id':'57702'},
                2018:{'season':380, 'l_id':'79022'},
                2017:{'season':371, 'l_id':'38584'},
                2016:{'season':359, 'l_id':'129773'},
                2015:{'season':348, 'l_id':'678764'},
                2014:{'season':331, 'l_id':'404819'}
}

#test that dictionary pulls correctly
print(seasonal_ids[2025]['l_id'])

#Check that types are coming correctly from dictionary
print(type(seasonal_ids[2025]['season'])) #needs to be int
print(type(seasonal_ids[2025]['l_id'])) #needs to be string

254906
<class 'int'>
<class 'str'>


## Start loading data

### In exploring documentation, YFPY queries work as follows -- items and bold will be adresssed first

- Base query initiated above (yahoo_query) has .some_query_name added to it for pulls from yahoo API
    - Each request needs season ID and league ID specific to that year
    - There are more than 20 separate queries in YFPY
    - For player statistics, unique player ID needs to be provided as well
        - For 10 years worth of weekly player data, the ultimate goal, this is approx. 250,000 requests
        - The YFPY get_player_stats query only allows one player ID at a time
        - Yahoo Fantasy API has a rate limit by the minute, hour and day
    - Explored yahoo_query.query()
        - This is a more rudimentary method that pulls directly from the yahoo API URL
            - Pulling directly from the URL allows for batching, which would work around rate limit
                - Yahoo API has a batch limit of 25 per request
                - This allows for automation of data pulling
- Files save as a JSON like file
    - Needs to be parsed correctly, though even the direct query.query call has been cleaned up a bit for more efficient loops
    - Need to automate converting over to dataframes and CSV files for storage and analysis
    - This requires using YFPY data save tool, with a conversion back to CSV with json_normalize


### Progress with first pulling every player for every season back to 2014, to obtain proper player IDs
First step is to pull a partial season's data and see what the files truly look like


In [6]:
# Run query to pull all 2025 season players as JSON
test_2025 = data.save('2025_test', #filename
          yahoo_query.get_league_players # call get league players query using info in base query
          )



2025-10-24 08:27:14.874 - ERROR - query.py - yfpy.query:541 - No data found when attempting extraction from fields: ['league', 'players']


In [7]:
# Print check
print(test_2025)

[Player({
  "bye_weeks": {
    "week": 5
  },
  "display_position": "QB",
  "editorial_player_key": "nfl.p.7200",
  "editorial_team_abbr": "Pit",
  "editorial_team_full_name": "Pittsburgh Steelers",
  "editorial_team_key": "nfl.t.23",
  "editorial_team_url": "https://sports.yahoo.com/nfl/teams/pittsburgh/",
  "eligible_positions": [
    "QB"
  ],
  "eligible_positions_to_add": null,
  "has_player_notes": 1,
  "headshot": {
    "size": "small",
    "url": "https://s.yimg.com/iu/api/res/1.2/Oj.vpAoZybhxtVc2aGTr6A--~C/YXBwaWQ9eXNwb3J0cztjaD0yMzM2O2NyPTE7Y3c9MTc5MDtkeD04NTc7ZHk9MDtmaT11bGNyb3A7aD02MDtxPTEwMDt3PTQ2/https://s.yimg.com/xe/i/us/sp/v/nfl_cutout/players_l/08202025/7200.png"
  },
  "image_url": "https://s.yimg.com/iu/api/res/1.2/Oj.vpAoZybhxtVc2aGTr6A--~C/YXBwaWQ9eXNwb3J0cztjaD0yMzM2O2NyPTE7Y3c9MTc5MDtkeD04NTc7ZHk9MDtmaT11bGNyb3A7aD02MDtxPTEwMDt3PTQ2/https://s.yimg.com/xe/i/us/sp/v/nfl_cutout/players_l/08202025/7200.png",
  "is_keeper": {
    "status": false,
    "cost": false,
 

In [8]:
# Data is in large JSON structure. Need to normalize into a version that is readable in dataframe

f = open('output/2025_test.json', 'r')
file = json.loads(f.read())
df = pd.json_normalize(file, sep='.')
df.head()

,player.bye_weeks.week,player.display_position,player.editorial_player_key,player.editorial_team_abbr,player.editorial_team_full_name,player.editorial_team_key,player.editorial_team_url,player.eligible_positions,player.eligible_positions_to_add,player.has_player_notes,player.headshot.size,player.headshot.url,player.image_url,player.is_keeper.status,player.is_keeper.cost,player.is_keeper.kept,player.is_undroppable,player.name.ascii_first,player.name.ascii_last,player.name.first,player.name.full,player.name.last,player.player_id,player.player_key,player.player_notes_last_timestamp,player.position_type,player.primary_position,player.uniform_number,player.url,player.status,player.status_full,player.has_recent_player_notes,player.injury_note
0,5,QB,nfl.p.7200,Pit,Pittsburgh Steelers,nfl.t.23,https://sports.yahoo.com/nfl/teams/pittsburgh/,[QB],None,1.0,small,https://s.yimg.com/iu/api/res/1.2/Oj.vpAoZybhx...,https://s.yimg.com/iu/api/res/1.2/Oj.vpAoZybhx...,False,False,False,0,Aaron,Rodgers,Aaron,Aaron Rodgers,Rodgers,7200,461.p.7200,1.760674e+09,O,QB,8,https://sports.yahoo.com/nfl/players/7200,NaN,NaN,NaN,NaN
1,5,TE,nfl.p.7777,Chi,Chicago Bears,nfl.t.3,https://sports.yahoo.com/nfl/teams/chicago/,[TE],None,NaN,small,https://s.yimg.com/iu/api/res/1.2/aou5CR6EOCjz...,https://s.yimg.com/iu/api/res/1.2/aou5CR6EOCjz...,False,False,False,0,Marcedes,Lewis,Marcedes,Marcedes Lewis,Lewis,7777,461.p.7777,NaN,O,TE,None,https://sports.yahoo.com/nfl/players/7777,NA,Inactive: Coach's Decision or Not on Roster,NaN,NaN
2,9,K,nfl.p.8432,NYJ,New York Jets,nfl.t.20,https://sports.yahoo.com/nfl/teams/ny-jets/,[K],None,1.0,small,https://s.yimg.com/iu/api/res/1.2/3.sAZrKS11dC...,https://s.yimg.com/iu/api/res/1.2/3.sAZrKS11dC...,False,False,False,0,Nick,Folk,Nick,Nick Folk,Folk,8432,461.p.8432,1.760927e+09,K,K,6,https://sports.yahoo.com/nfl/players/8432,NaN,NaN,NaN,NaN
3,7,K,nfl.p.8565,Buf,Buffalo Bills,nfl.t.2,https://sports.yahoo.com/nfl/teams/buffalo/,[K],None,NaN,small,https://s.yimg.com/iu/api/res/1.2/0A0dIdoRxUxO...,https://s.yimg.com/iu/api/res/1.2/0A0dIdoRxUxO...,False,False,False,0,Matt,Prater,Matt,Matt Prater,Prater,8565,461.p.8565,NaN,K,K,15,https://sports.yahoo.com/nfl/players/8565,NaN,NaN,NaN,NaN
4,10,QB,nfl.p.8795,Cin,Cincinnati Bengals,nfl.t.4,https://sports.yahoo.com/nfl/teams/cincinnati/,[QB],None,1.0,small,https://s.yimg.com/iu/api/res/1.2/vhO8VjyUIfV6...,https://s.yimg.com/iu/api/res/1.2/vhO8VjyUIfV6...,False,False,False,0,Joe,Flacco,Joe,Joe Flacco,Flacco,8795,461.p.8795,1.760674e+09,O,QB,16,https://sports.yahoo.com/nfl/players/8795,NaN,NaN,NaN,NaN


In [9]:
# how many rows of data/players in 2025?

df['player.editorial_player_key'].describe()

count           1248
unique          1248
top       nfl.p.7200
freq               1
Name: player.editorial_player_key, dtype: object

Seeing as each team has 12 active players on offense, 1248 unique players is more than enough

#### Loop through seasons to obtain all years of players

In [10]:
# Create function to allow looping over seasonal_ids dictionary to obtain all player IDs

def saver(season, query_name, file_pre):
    """
    Function takes season, yahoo query name and file output type, parses JSON and saves to CSV
    
    Args:
        season (int): Calendar year of season, 2014 thru 2025 available
        query_name (str): subquery to attach to yahoo base query, options found in YFPY documentation
        file_pre (str): prefix to use for naming files (i.e player, player_stats, team_stats, etc.)
    
    Returns:

    Saves CSV files and prints (season success! when CSV downloads)
    """
    season_for_name = str(season) # save season as string for filename
    yahoo_query = YahooFantasySportsQuery(
        league_id=seasonal_ids[season]['l_id'], # pulls correct league ID for season entered
        game_code="nfl",
        game_id=seasonal_ids[season]['season'], # pulls correct season ID for season entered
        yahoo_consumer_key=os.getenv('CONS_KEY'),
        yahoo_consumer_secret=os.getenv('CONS_SECRET'),
        env_file_location=Path('.'),
        env_var_fallback=True,
        save_token_data_to_env_file=True
        )
    full_query = getattr(yahoo_query, query_name) #need to dynamically apply method from 'query name' entered in function, getattr pulls attribute for use
    filename = f'{file_pre}_{season_for_name}' # add file pre and season to file
    data.save(filename, full_query) # save query as JSON
    f = open(f'output/{filename}.json', 'r')
    file = json.loads(f.read())
    df = pd.json_normalize(file, sep='.')
    df.to_csv(f'{filename}.csv')
    print(f'{season_for_name} success!')

**THE FOLLOWING CODE BLOCK IS HASHED OUT TO AVOID RUNNING THE LOOP AGAIN**

In [11]:
# #Create loop to go over all seasons

# for season in seasonal_ids:
#     try:
#         print(f'Processing {season} season')
#         saver(season, 'get_league_players', 'players')
#         time.sleep(5) # to avoid disconnection or rate limit
#     except Exception as e:
#         print(f'Error with {season} season: {e}')
#         continue

In [12]:
# Create loop to grab all years and convert to single dataframe
dfs = []
seasons = list(range(2014,2026))

for season in seasons:
    df = pd.read_csv(filepath_or_buffer=f'Players/players_{season}.csv',)
    df['season'] = season # add season column for later use
    dfs.append(df)


In [13]:

# concatenate all dataframes held in list
all_players = pd.concat(dfs, ignore_index=True)


In [14]:
# Check that the number if entries seems reasonable
all_players.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14096 entries, 0 to 14095
Data columns (total 62 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   Unnamed: 0                             14096 non-null  int64  
 1   player.bye_weeks.week                  14096 non-null  int64  
 2   player.display_position                14096 non-null  object 
 3   player.editorial_player_key            14096 non-null  object 
 4   player.editorial_team_abbr             14096 non-null  object 
 5   player.editorial_team_full_name        14096 non-null  object 
 6   player.editorial_team_key              14096 non-null  object 
 7   player.editorial_team_url              14096 non-null  object 
 8   player.eligible_positions              14096 non-null  object 
 9   player.eligible_positions_to_add       0 non-null      float64
 10  player.headshot.size                   14096 non-null  object 
 11  pl

#### Clean dataframe

In [15]:
# Drop old index column

all_players = all_players.drop(columns='Unnamed: 0')
all_players.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14096 entries, 0 to 14095
Data columns (total 61 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   player.bye_weeks.week                  14096 non-null  int64  
 1   player.display_position                14096 non-null  object 
 2   player.editorial_player_key            14096 non-null  object 
 3   player.editorial_team_abbr             14096 non-null  object 
 4   player.editorial_team_full_name        14096 non-null  object 
 5   player.editorial_team_key              14096 non-null  object 
 6   player.editorial_team_url              14096 non-null  object 
 7   player.eligible_positions              14096 non-null  object 
 8   player.eligible_positions_to_add       0 non-null      float64
 9   player.headshot.size                   14096 non-null  object 
 10  player.headshot.url                    14096 non-null  object 
 11  pl

In [16]:
# Rename columns for clarity
all_players.columns = all_players.columns.str.replace('player.','')

# Remove columns aith all nulls
all_players = all_players.dropna(axis=1, how='all')

#info check
all_players.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14096 entries, 0 to 14095
Data columns (total 33 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   bye_weeks.week               14096 non-null  int64  
 1   display_position             14096 non-null  object 
 2   editorial_player_key         14096 non-null  object 
 3   editorial_team_abbr          14096 non-null  object 
 4   editorial_team_full_name     14096 non-null  object 
 5   editorial_team_key           14096 non-null  object 
 6   editorial_team_url           14096 non-null  object 
 7   eligible_positions           14096 non-null  object 
 8   headshot.size                14096 non-null  object 
 9   headshot.url                 14096 non-null  object 
 10  image_url                    14096 non-null  object 
 11  is_keeper.status             14096 non-null  bool   
 12  is_keeper.cost               14096 non-null  bool   
 13  is_keeper.kept  

In [17]:
#Check for missing last namees, doesn't make sense
missing_last = all_players[all_players['name.ascii_last'].isna()]

#last names look to be defenses, check values of display position
print(missing_last['display_position'].value_counts())

# missing last is defenses, check what full name says
print(f'\nDefense full name, defense first name: \n{missing_last[["name.ascii_first", "name.full"]].head()}')

# Check editorial names for teams
print(f'\nEditorial Names: \n{missing_last["editorial_team_full_name"].head()}')

# Make mascot the last name for defense
all_players.loc[all_players['name.last'].isna(), 'name.last'] = all_players['editorial_team_full_name'].str.split().str[-1]

# Make editorial name the full name for defnese
all_players.loc[all_players['display_position'] == 'DEF', 'name.full'] = all_players['editorial_team_full_name'].astype(str)

print('\ncheck work after cleaning defense names')
display(all_players[all_players['display_position'] =='DEF'].head())



display_position
DEF    384
Name: count, dtype: int64

Defense full name, defense first name: 
     name.ascii_first   name.full
1140          Atlanta     Atlanta
1141          Buffalo     Buffalo
1142          Chicago     Chicago
1143       Cincinnati  Cincinnati
1144        Cleveland   Cleveland

Editorial Names: 
1140       Atlanta Falcons
1141         Buffalo Bills
1142         Chicago Bears
1143    Cincinnati Bengals
1144      Cleveland Browns
Name: editorial_team_full_name, dtype: object

check work after cleaning defense names


,bye_weeks.week,display_position,editorial_player_key,editorial_team_abbr,editorial_team_full_name,editorial_team_key,editorial_team_url,eligible_positions,headshot.size,headshot.url,image_url,is_keeper.status,is_keeper.cost,is_keeper.kept,is_undroppable,name.ascii_first,name.ascii_last,name.first,name.full,name.last,player_id,player_key,position_type,primary_position,uniform_number,url,has_player_notes,player_notes_last_timestamp,has_recent_player_notes,season,injury_note,status,status_full
1140,9,DEF,nfl.p.100001,Atl,Atlanta Falcons,nfl.t.1,https://sports.yahoo.com/nfl/teams/atlanta/,['DEF'],small,https://s.yimg.com/lq/i/us/sp/v/nfl/teams/1/50...,https://s.yimg.com/lq/i/us/sp/v/nfl/teams/1/50...,False,False,False,0,Atlanta,NaN,Atlanta,Atlanta Falcons,Falcons,100001,331.p.100001,DT,DEF,False,https://sports.yahoo.com/nfl/teams/atlanta/,NaN,NaN,NaN,2014,NaN,NaN,NaN
1141,9,DEF,nfl.p.100002,Buf,Buffalo Bills,nfl.t.2,https://sports.yahoo.com/nfl/teams/buffalo/,['DEF'],small,https://s.yimg.com/lq/i/us/sp/v/nfl/teams/1/50...,https://s.yimg.com/lq/i/us/sp/v/nfl/teams/1/50...,False,False,False,0,Buffalo,NaN,Buffalo,Buffalo Bills,Bills,100002,331.p.100002,DT,DEF,False,https://sports.yahoo.com/nfl/teams/buffalo/,NaN,NaN,NaN,2014,NaN,NaN,NaN
1142,9,DEF,nfl.p.100003,Chi,Chicago Bears,nfl.t.3,https://sports.yahoo.com/nfl/teams/chicago/,['DEF'],small,https://s.yimg.com/lq/i/us/sp/v/nfl/teams/1/50...,https://s.yimg.com/lq/i/us/sp/v/nfl/teams/1/50...,False,False,False,0,Chicago,NaN,Chicago,Chicago Bears,Bears,100003,331.p.100003,DT,DEF,False,https://sports.yahoo.com/nfl/teams/chicago/,NaN,NaN,NaN,2014,NaN,NaN,NaN
1143,4,DEF,nfl.p.100004,Cin,Cincinnati Bengals,nfl.t.4,https://sports.yahoo.com/nfl/teams/cincinnati/,['DEF'],small,https://s.yimg.com/lq/i/us/sp/v/nfl/teams/1/50...,https://s.yimg.com/lq/i/us/sp/v/nfl/teams/1/50...,False,False,False,0,Cincinnati,NaN,Cincinnati,Cincinnati Bengals,Bengals,100004,331.p.100004,DT,DEF,False,https://sports.yahoo.com/nfl/teams/cincinnati/,NaN,NaN,NaN,2014,NaN,NaN,NaN
1144,4,DEF,nfl.p.100005,Cle,Cleveland Browns,nfl.t.5,https://sports.yahoo.com/nfl/teams/cleveland/,['DEF'],small,https://s.yimg.com/lq/i/us/sp/v/nfl/teams/1/50...,https://s.yimg.com/lq/i/us/sp/v/nfl/teams/1/50...,False,False,False,0,Cleveland,NaN,Cleveland,Cleveland Browns,Browns,100005,331.p.100005,DT,DEF,False,https://sports.yahoo.com/nfl/teams/cleveland/,NaN,NaN,NaN,2014,NaN,NaN,NaN


In [18]:
# another info check
all_players.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14096 entries, 0 to 14095
Data columns (total 33 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   bye_weeks.week               14096 non-null  int64  
 1   display_position             14096 non-null  object 
 2   editorial_player_key         14096 non-null  object 
 3   editorial_team_abbr          14096 non-null  object 
 4   editorial_team_full_name     14096 non-null  object 
 5   editorial_team_key           14096 non-null  object 
 6   editorial_team_url           14096 non-null  object 
 7   eligible_positions           14096 non-null  object 
 8   headshot.size                14096 non-null  object 
 9   headshot.url                 14096 non-null  object 
 10  image_url                    14096 non-null  object 
 11  is_keeper.status             14096 non-null  bool   
 12  is_keeper.cost               14096 non-null  bool   
 13  is_keeper.kept  

In [19]:
# Remaining columns with missing values are not of any value, will drop
all_players = all_players.dropna(axis=1)
all_players.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14096 entries, 0 to 14095
Data columns (total 25 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   bye_weeks.week            14096 non-null  int64 
 1   display_position          14096 non-null  object
 2   editorial_player_key      14096 non-null  object
 3   editorial_team_abbr       14096 non-null  object
 4   editorial_team_full_name  14096 non-null  object
 5   editorial_team_key        14096 non-null  object
 6   editorial_team_url        14096 non-null  object
 7   eligible_positions        14096 non-null  object
 8   headshot.size             14096 non-null  object
 9   headshot.url              14096 non-null  object
 10  image_url                 14096 non-null  object
 11  is_keeper.status          14096 non-null  bool  
 12  is_keeper.cost            14096 non-null  bool  
 13  is_keeper.kept            14096 non-null  bool  
 14  is_undroppable        

In [20]:
# # All NaNs are handled, dump to csv
# all_players.to_csv(path_or_buf='Players/players_raw_14to25.csv')


### Next, gather all player IDs together to feed into yahoo query

In [21]:
# How many uniaue keys compared to number of lines
print(f'all lines: {len(all_players["player_key"])}, unique player IDs: {all_players["player_key"].nunique()}')

all lines: 14096, unique player IDs: 14096


In [22]:
# We have al unique items, convert keys to list for batch use in next query
p_id_list = list(all_players['player_key'])
print(f'the number of items in the player key list is {len(p_id_list)} \nthe first 10 items in p_id_list are {p_id_list[:10]}')

the number of items in the player key list is 14096 
the first 10 items in p_id_list are ['331.p.3727', '331.p.3950', '331.p.4256', '331.p.4269', '331.p.4416', '331.p.4587', '331.p.5046', '331.p.5228', '331.p.5381', '331.p.5388']


In [23]:
seasonal_keys = {}
for season in seasonal_ids:
    one_season = all_players[all_players['season'] == season]
    seasonal_keys[season] = list(one_season['player_key'])

display(seasonal_keys)    

{2025: ['461.p.7200',
  '461.p.7777',
  '461.p.8432',
  '461.p.8565',
  '461.p.8795',
  '461.p.8937',
  '461.p.9265',
  '461.p.9526',
  '461.p.9547',
  '461.p.24070',
  '461.p.24797',
  '461.p.24822',
  '461.p.24851',
  '461.p.24967',
  '461.p.25718',
  '461.p.25785',
  '461.p.25812',
  '461.p.25871',
  '461.p.25876',
  '461.p.25881',
  '461.p.26389',
  '461.p.26483',
  '461.p.26534',
  '461.p.26650',
  '461.p.26652',
  '461.p.26658',
  '461.p.26662',
  '461.p.26664',
  '461.p.26686',
  '461.p.26699',
  '461.p.26701',
  '461.p.26721',
  '461.p.26753',
  '461.p.26800',
  '461.p.26804',
  '461.p.27120',
  '461.p.27277',
  '461.p.27369',
  '461.p.27535',
  '461.p.27548',
  '461.p.27560',
  '461.p.27564',
  '461.p.27570',
  '461.p.27581',
  '461.p.27589',
  '461.p.27590',
  '461.p.27624',
  '461.p.27648',
  '461.p.27692',
  '461.p.28026',
  '461.p.28115',
  '461.p.28141',
  '461.p.28188',
  '461.p.28227',
  '461.p.28365',
  '461.p.28378',
  '461.p.28389',
  '461.p.28390',
  '461.p.28392',


In [24]:
seasonal_keys[2024]

['449.p.7200',
 '449.p.7777',
 '449.p.8432',
 '449.p.8447',
 '449.p.8565',
 '449.p.8795',
 '449.p.8937',
 '449.p.9265',
 '449.p.9526',
 '449.p.24070',
 '449.p.24793',
 '449.p.24797',
 '449.p.24822',
 '449.p.24851',
 '449.p.24967',
 '449.p.25718',
 '449.p.25785',
 '449.p.25812',
 '449.p.25871',
 '449.p.25881',
 '449.p.26389',
 '449.p.26483',
 '449.p.26534',
 '449.p.26650',
 '449.p.26652',
 '449.p.26658',
 '449.p.26662',
 '449.p.26664',
 '449.p.26686',
 '449.p.26699',
 '449.p.26701',
 '449.p.26721',
 '449.p.26753',
 '449.p.26800',
 '449.p.26804',
 '449.p.27120',
 '449.p.27277',
 '449.p.27369',
 '449.p.27535',
 '449.p.27540',
 '449.p.27548',
 '449.p.27560',
 '449.p.27564',
 '449.p.27570',
 '449.p.27581',
 '449.p.27589',
 '449.p.27590',
 '449.p.27624',
 '449.p.27646',
 '449.p.27648',
 '449.p.28026',
 '449.p.28141',
 '449.p.28188',
 '449.p.28227',
 '449.p.28365',
 '449.p.28378',
 '449.p.28389',
 '449.p.28390',
 '449.p.28392',
 '449.p.28402',
 '449.p.28408',
 '449.p.28417',
 '449.p.28442',
 

#### This is the portion that requires the direct query to yahoo's api to batch requests. First attempt is to pull three

In [25]:
# get last three keys on player list
three = p_id_list[:3]
print(three)

# print ids dictionary for reference
display(seasonal_ids)

['331.p.3727', '331.p.3950', '331.p.4256']


{2025: {'season': 461, 'l_id': '254906'},
 2024: {'season': 449, 'l_id': '17240'},
 2023: {'season': 423, 'l_id': '541098'},
 2022: {'season': 414, 'l_id': '21504'},
 2021: {'season': 406, 'l_id': '8243'},
 2020: {'season': 399, 'l_id': '47784'},
 2019: {'season': 390, 'l_id': '57702'},
 2018: {'season': 380, 'l_id': '79022'},
 2017: {'season': 371, 'l_id': '38584'},
 2016: {'season': 359, 'l_id': '129773'},
 2015: {'season': 348, 'l_id': '678764'},
 2014: {'season': 331, 'l_id': '404819'}}

In [26]:
# Combine three items into one string with no spaces
test = ",".join(three)
print(test, type(test))


331.p.3727,331.p.3950,331.p.4256 <class 'str'>


In [27]:
# Try direct query before automating

query_test = YahooFantasySportsQuery(
        league_id=404819, # pulls correct league ID for season entered
        game_code="nfl",
        yahoo_consumer_key=os.getenv('CONS_KEY'),
        yahoo_consumer_secret=os.getenv('CONS_SECRET'),
        env_file_location=Path('.'),
        env_var_fallback=True,
        save_token_data_to_env_file=True
        )


direct_test = query_test.query(url = f'https://fantasysports.yahooapis.com/fantasy/v2/league/331.l.404819/players;player_keys={test}/stats', data_key_list=['league', 'players'])
print(direct_test)

[Player({
  "bye_weeks": {
    "week": 10
  },
  "display_position": "K",
  "editorial_player_key": "nfl.p.3727",
  "editorial_team_abbr": "Ind",
  "editorial_team_full_name": "Indianapolis Colts",
  "editorial_team_key": "nfl.t.11",
  "editorial_team_url": "https://sports.yahoo.com/nfl/teams/indianapolis/",
  "eligible_positions": [
    "K"
  ],
  "eligible_positions_to_add": null,
  "headshot": {
    "size": "small",
    "url": "https://s.yimg.com/iu/api/res/1.2/OpHvpCHjl_PQvkeQUgsjsA--~C/YXBwaWQ9eXNwb3J0cztjaD0yMzM2O2NyPTE7Y3c9MTc5MDtkeD04NTc7ZHk9MDtmaT11bGNyb3A7aD02MDtxPTEwMDt3PTQ2/https://s.yimg.com/xe/i/us/sp/v/nfl_cutout/players_l/08152019/3727.png"
  },
  "image_url": "https://s.yimg.com/iu/api/res/1.2/OpHvpCHjl_PQvkeQUgsjsA--~C/YXBwaWQ9eXNwb3J0cztjaD0yMzM2O2NyPTE7Y3c9MTc5MDtkeD04NTc7ZHk9MDtmaT11bGNyb3A7aD02MDtxPTEwMDt3PTQ2/https://s.yimg.com/xe/i/us/sp/v/nfl_cutout/players_l/08152019/3727.png",
  "is_keeper": {
    "status": false,
    "cost": false,
    "kept": false
  },
  "

In [28]:
def direct_query(player_keys, file_pre, batch_size):
    for season in seasonal_ids:
        dfs = []
        yahoo_query = YahooFantasySportsQuery(
        league_id=seasonal_ids[season]['l_id'], # pulls correct league ID for season entered
        game_code="nfl",
        yahoo_consumer_key=os.getenv('CONS_KEY'),
        yahoo_consumer_secret=os.getenv('CONS_SECRET'),
        env_file_location=Path('.'),
        env_var_fallback=True,
        save_token_data_to_env_file=True
        )
        seasonal_player_keys = player_keys[season]
        season_for_name = str(season)
        for i in range(0, len(seasonal_player_keys), batch_size):
            total_batches = (len(seasonal_player_keys) - 1) // batch_size + 1
            batch_num = i // batch_size + 1
            print(f'Processing batch {batch_num} of {total_batches} in {season}')
            batch = seasonal_player_keys[i:i+batch_size]
            batch_string = ','.join(batch)
            filename = f'{file_pre}_{season_for_name}' # add file pre and season to file
            data.save(filename, yahoo_query.query,
                    params = {
                        'url' : f'https://fantasysports.yahooapis.com/fantasy/v2/league/{seasonal_ids[season]["season"]}.l.{seasonal_ids[season]["l_id"]}/players;player_keys={batch_string}/stats',
                        'data_key_list' : ['league', 'players']
                    })
            f = open(f'output/{filename}.json', 'r')
            file = json.loads(f.read())
            df = pd.json_normalize(file, sep='.')
            dfs.append(df)
            time.sleep(5)
        all_players = pd.concat(dfs, ignore_index=True)
        all_players.to_csv(f'{filename}.csv')
        print(f'{season_for_name} success!')



In [29]:
# direct_query(seasonal_keys, 'stats', 25)

In [30]:
# Create loop to grab all years and convert to single dataframe
dfs = []
seasons = list(range(2014,2026))

for season in seasons:
    df = pd.read_csv(filepath_or_buffer=f'Stats/Yahoo/stats_{season}.csv',)
    dfs.append(df)

In [31]:
# concatenate all dataframes held in list
all_stats = pd.concat(dfs, ignore_index=True)

In [32]:
all_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15344 entries, 0 to 15343
Data columns (total 43 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   Unnamed: 0                                  15344 non-null  int64  
 1   player.bye_weeks.week                       15344 non-null  int64  
 2   player.display_position                     15344 non-null  object 
 3   player.editorial_player_key                 15344 non-null  object 
 4   player.editorial_team_abbr                  15344 non-null  object 
 5   player.editorial_team_full_name             15344 non-null  object 
 6   player.editorial_team_key                   15344 non-null  object 
 7   player.editorial_team_url                   15344 non-null  object 
 8   player.eligible_positions                   15344 non-null  object 
 9   player.eligible_positions_to_add            0 non-null      float64
 10  player.hea

In [33]:
all_stats = all_stats.drop(columns='Unnamed: 0')
# Rename columns for clarity
all_stats.columns = all_stats.columns.str.replace('player.','')

# Remove columns aith all nulls
all_stats = all_stats.dropna(axis=1, how='all')

#info check
all_stats.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15344 entries, 0 to 15343
Data columns (total 41 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   bye_weeks.week                       15344 non-null  int64  
 1   display_position                     15344 non-null  object 
 2   editorial_player_key                 15344 non-null  object 
 3   editorial_team_abbr                  15344 non-null  object 
 4   editorial_team_full_name             15344 non-null  object 
 5   editorial_team_key                   15344 non-null  object 
 6   editorial_team_url                   15344 non-null  object 
 7   eligible_positions                   15344 non-null  object 
 8   headshot.size                        15344 non-null  object 
 9   headshot.url                         15344 non-null  object 
 10  image_url                            15344 non-null  object 
 11  is_keeper.status            

In [34]:

#Check for missing last namees, doesn't make sense
missing_last = all_stats[all_stats['name.ascii_last'].isna()]

#last names look to be defenses, check values of display position
print(missing_last['display_position'].value_counts())

# missing last is defenses, check what full name says
print(f'\nDefense full name, defense first name: \n{missing_last[["name.ascii_first", "name.full"]].head()}')

# Check editorial names for teams
print(f'\nEditorial Names: \n{missing_last["editorial_team_full_name"].head()}')


display_position
DEF    416
Name: count, dtype: int64

Defense full name, defense first name: 
     name.ascii_first   name.full
1140          Atlanta     Atlanta
1141          Buffalo     Buffalo
1142          Chicago     Chicago
1143       Cincinnati  Cincinnati
1144        Cleveland   Cleveland

Editorial Names: 
1140       Atlanta Falcons
1141         Buffalo Bills
1142         Chicago Bears
1143    Cincinnati Bengals
1144      Cleveland Browns
Name: editorial_team_full_name, dtype: object


In [35]:

# Make mascot the last name for defense
all_stats.loc[all_stats['name.last'].isna(), 'name.last'] = all_stats['editorial_team_full_name'].str.split().str[-1]

# Make editorial name the full name for defnese
all_stats.loc[all_stats['display_position'] == 'DEF', 'name.full'] = all_stats['editorial_team_full_name'].astype(str)

print('\ncheck work after cleaning defense names')
display(all_stats[all_stats['display_position'] =='DEF'].head())


check work after cleaning defense names


,bye_weeks.week,display_position,editorial_player_key,editorial_team_abbr,editorial_team_full_name,editorial_team_key,editorial_team_url,eligible_positions,headshot.size,headshot.url,image_url,is_keeper.status,is_keeper.cost,is_keeper.kept,is_undroppable,name.ascii_first,name.ascii_last,name.first,name.full,name.last,player_id,player_key,player_points.coverage_type,player_points.season,player_points.total,player_stats.coverage_type,player_stats.season,player_stats.stats,position_type,primary_position,uniform_number,url,has_player_notes,player_notes_last_timestamp,has_recent_player_notes,injury_note,status,status_full,player_advanced_stats.coverage_type,player_advanced_stats.season,player_advanced_stats.stats
1140,9,DEF,nfl.p.100001,Atl,Atlanta Falcons,nfl.t.1,https://sports.yahoo.com/nfl/teams/atlanta/,['DEF'],small,https://s.yimg.com/lq/i/us/sp/v/nfl/teams/1/50...,https://s.yimg.com/lq/i/us/sp/v/nfl/teams/1/50...,False,False,False,0,Atlanta,NaN,Atlanta,Atlanta Falcons,Falcons,100001,331.p.100001,season,2014,144.76,season,2014,"[{'stat': {'stat_id': 31, 'value': 0.0}}, {'st...",DT,DEF,False,https://sports.yahoo.com/nfl/teams/atlanta/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1141,9,DEF,nfl.p.100002,Buf,Buffalo Bills,nfl.t.2,https://sports.yahoo.com/nfl/teams/buffalo/,['DEF'],small,https://s.yimg.com/lq/i/us/sp/v/nfl/teams/1/50...,https://s.yimg.com/lq/i/us/sp/v/nfl/teams/1/50...,False,False,False,0,Buffalo,NaN,Buffalo,Buffalo Bills,Bills,100002,331.p.100002,season,2014,271.48,season,2014,"[{'stat': {'stat_id': 31, 'value': 0.0}}, {'st...",DT,DEF,False,https://sports.yahoo.com/nfl/teams/buffalo/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1142,9,DEF,nfl.p.100003,Chi,Chicago Bears,nfl.t.3,https://sports.yahoo.com/nfl/teams/chicago/,['DEF'],small,https://s.yimg.com/lq/i/us/sp/v/nfl/teams/1/50...,https://s.yimg.com/lq/i/us/sp/v/nfl/teams/1/50...,False,False,False,0,Chicago,NaN,Chicago,Chicago Bears,Bears,100003,331.p.100003,season,2014,162.98,season,2014,"[{'stat': {'stat_id': 31, 'value': 0.0}}, {'st...",DT,DEF,False,https://sports.yahoo.com/nfl/teams/chicago/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1143,4,DEF,nfl.p.100004,Cin,Cincinnati Bengals,nfl.t.4,https://sports.yahoo.com/nfl/teams/cincinnati/,['DEF'],small,https://s.yimg.com/lq/i/us/sp/v/nfl/teams/1/50...,https://s.yimg.com/lq/i/us/sp/v/nfl/teams/1/50...,False,False,False,0,Cincinnati,NaN,Cincinnati,Cincinnati Bengals,Bengals,100004,331.p.100004,season,2014,154.54,season,2014,"[{'stat': {'stat_id': 31, 'value': 0.0}}, {'st...",DT,DEF,False,https://sports.yahoo.com/nfl/teams/cincinnati/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1144,4,DEF,nfl.p.100005,Cle,Cleveland Browns,nfl.t.5,https://sports.yahoo.com/nfl/teams/cleveland/,['DEF'],small,https://s.yimg.com/lq/i/us/sp/v/nfl/teams/1/50...,https://s.yimg.com/lq/i/us/sp/v/nfl/teams/1/50...,False,False,False,0,Cleveland,NaN,Cleveland,Cleveland Browns,Browns,100005,331.p.100005,season,2014,181.64,season,2014,"[{'stat': {'stat_id': 31, 'value': 0.0}}, {'st...",DT,DEF,False,https://sports.yahoo.com/nfl/teams/cleveland/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
# Info check
all_stats.info()

# get remaining columns with missing values
all_stats.isna().sum()[all_stats.isna().sum() > 0]



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15344 entries, 0 to 15343
Data columns (total 41 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   bye_weeks.week                       15344 non-null  int64  
 1   display_position                     15344 non-null  object 
 2   editorial_player_key                 15344 non-null  object 
 3   editorial_team_abbr                  15344 non-null  object 
 4   editorial_team_full_name             15344 non-null  object 
 5   editorial_team_key                   15344 non-null  object 
 6   editorial_team_url                   15344 non-null  object 
 7   eligible_positions                   15344 non-null  object 
 8   headshot.size                        15344 non-null  object 
 9   headshot.url                         15344 non-null  object 
 10  image_url                            15344 non-null  object 
 11  is_keeper.status            

name.ascii_last                          416
uniform_number                          3843
has_player_notes                       12429
player_notes_last_timestamp            12429
has_recent_player_notes                14785
injury_note                            13613
status                                 14764
status_full                             5303
player_advanced_stats.coverage_type     8821
player_advanced_stats.season            8821
player_advanced_stats.stats             8821
dtype: int64

In [37]:
# Remaining columns with missing values are not of any value, will drop
all_stats = all_stats.dropna(axis=1)
all_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15344 entries, 0 to 15343
Data columns (total 30 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   bye_weeks.week               15344 non-null  int64  
 1   display_position             15344 non-null  object 
 2   editorial_player_key         15344 non-null  object 
 3   editorial_team_abbr          15344 non-null  object 
 4   editorial_team_full_name     15344 non-null  object 
 5   editorial_team_key           15344 non-null  object 
 6   editorial_team_url           15344 non-null  object 
 7   eligible_positions           15344 non-null  object 
 8   headshot.size                15344 non-null  object 
 9   headshot.url                 15344 non-null  object 
 10  image_url                    15344 non-null  object 
 11  is_keeper.status             15344 non-null  bool   
 12  is_keeper.cost               15344 non-null  bool   
 13  is_keeper.kept  

There should be the same number of records here as in the first dataset, check for duplicates

In [38]:
# Dupe check, see if theres a common issue basd on season
dupes = all_stats[all_stats.duplicated(keep=False)].sort_values('player_key')
dupes['player_stats.season'].value_counts()

player_stats.season
2025    2496
Name: count, dtype: int64

In [39]:
# One season was double imported, ok to drop dupes

all_stats = all_stats.drop_duplicates(keep='last')
all_stats.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14096 entries, 0 to 15343
Data columns (total 30 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   bye_weeks.week               14096 non-null  int64  
 1   display_position             14096 non-null  object 
 2   editorial_player_key         14096 non-null  object 
 3   editorial_team_abbr          14096 non-null  object 
 4   editorial_team_full_name     14096 non-null  object 
 5   editorial_team_key           14096 non-null  object 
 6   editorial_team_url           14096 non-null  object 
 7   eligible_positions           14096 non-null  object 
 8   headshot.size                14096 non-null  object 
 9   headshot.url                 14096 non-null  object 
 10  image_url                    14096 non-null  object 
 11  is_keeper.status             14096 non-null  bool   
 12  is_keeper.cost               14096 non-null  bool   
 13  is_keeper.kept       

In [40]:
# All NaNs and dupes are handled, dump to csv

# all_stats.to_csv(path_or_buf='Raw/stats_yahooleague_raw_14to25.csv')


# Work with nflreadpy data

- The yahoo fantasy API provides league specific season long scores for each player. This is important because our league has a soecific scoring system that is unique
    - However, the yahoo fantasy API does not track granular statistics (receptions, carries, etc) further back than a couple years
- SOLUTION -- Use nflreadpy, an expeansive libray pulling from many data sources, to come up with granular statistics for all players

In [41]:
# test recent season load converted to pandas

test_2025 = nfl.load_player_stats(seasons=2025, summary_level='reg').to_pandas()

In [42]:
# Check if DST is in these or if it is only Indiviudal positions

test_2025['position'].value_counts().sort_index()

position
C       29
CB     188
DB      11
DE     128
DL       5
DT     135
FB       5
FS       9
G       68
ILB      4
K       38
LB     227
LS      17
MLB      6
NT       6
OL       2
OLB     18
OT     100
P       35
QB      64
RB     131
S        6
SAF    133
TE     115
WR     203
Name: count, dtype: int64

Only individual defensive players are listed, team statistics are available in nfl data

In [43]:
team_2025 = nfl.load_team_stats(seasons=2024, summary_level='reg').to_pandas()
team_2025.head()

,season,team,season_type,games,completions,attempts,passing_yards,passing_tds,passing_interceptions,sacks_suffered,sack_yards_lost,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,passing_epa,passing_cpoe,passing_2pt_conversions,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_epa,rushing_2pt_conversions,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,special_teams_tds,def_tackles_solo,def_tackles_with_assist,def_tackle_assists,def_tackles_for_loss,def_tackles_for_loss_yards,def_fumbles_forced,def_sacks,def_sack_yards,def_qb_hits,def_interceptions,def_interception_yards,def_pass_defended,def_tds,def_fumbles,def_safeties,misc_yards,fumble_recovery_own,fumble_recovery_yards_own,fumble_recovery_opp,fumble_recovery_yards_opp,fumble_recovery_tds,penalties,penalty_yards,timeouts,punt_returns,punt_return_yards,kickoff_returns,kickoff_return_yards,fg_made,fg_att,fg_missed,fg_blocked,fg_long,fg_pct,fg_made_0_19,fg_made_20_29,fg_made_30_39,fg_made_40_49,fg_made_50_59,fg_made_60_,fg_missed_0_19,fg_missed_20_29,fg_missed_30_39,fg_missed_40_49,fg_missed_50_59,fg_missed_60_,fg_made_list,fg_missed_list,fg_blocked_list,fg_made_distance,fg_missed_distance,fg_blocked_distance,pat_made,pat_att,pat_missed,pat_blocked,pat_pct,gwfg_made,gwfg_att,gwfg_missed,gwfg_blocked,gwfg_distance_list
0,2024,ARI,REG,17,374,543,3859,21,11,30,-220,3,1,3697,1974,191,64.526349,2.181092,0,463,2451,18,9,4,147,21.713928,3,374,523,3859,21,5,4,3697,1972,191,109.596055,0,1,566,91,497,87,351,17,41.0,256.0,76,9,12,63,0,2,2,0,10,13,8,8,2,92,806,52,21,202,25,709,34,38,3,1,58,0.894737,0,7,12,10,5,0,0,0,0,3,0,0,29;31;48;57;42;45;30;42;35;38;40;32;57;34;29;5...,47;45;40,45,1304,132,45,36,37,0,1,0.972973,2,2,0,0,32;34
1,2024,ATL,REG,17,364,559,4283,21,19,32,-215,9,2,4466,1958,199,34.316687,-0.062625,0,495,2219,18,6,0,147,0.128415,2,364,539,4283,21,3,1,4466,1958,199,104.144791,0,0,628,67,445,79,307,12,31.0,187.0,74,12,187,66,3,0,1,0,13,19,6,6,1,99,828,64,21,200,30,791,29,41,10,2,58,0.707317,0,7,8,7,7,0,0,0,1,5,4,0,24;39;22;34;54;53;44;42;58;54;48;52;21;31;35;3...,41;54;46;53;46;35;42;43;56;52,54;35,1132,468,89,38,38,0,0,1.000000,1,1,0,0,58
2,2024,BAL,REG,17,318,477,4189,41,4,24,-154,2,1,4162,2086,199,170.468876,4.702599,0,554,3189,21,9,5,173,50.346863,1,318,457,4189,41,1,1,4162,2086,199,219.517794,0,0,591,61,419,82,362,12,53.0,314.0,122,12,102,82,2,3,0,0,12,17,5,5,1,132,1120,66,22,213,30,811,22,30,8,0,56,0.733333,0,4,7,5,6,0,0,0,0,3,5,0,25;32;48;42;32;56;24;45;32;39;28;52;49;33;37;5...,53;56;46;50;47;50;47;53,None,883,402,0,60,62,2,0,0.967742,1,1,0,0,24
3,2024,BUF,REG,17,329,520,3938,30,6,14,-63,4,2,4262,2186,181,132.671717,0.253748,1,491,2230,32,2,0,144,48.125277,0,329,495,3938,30,2,0,4262,2186,181,172.570791,1,0,604,101,425,93,426,18,37.0,246.0,94,16,145,60,2,1,1,0,8,8,16,16,1,112,833,68,27,313,24,621,24,29,5,0,61,0.827586,0,6,7,7,3,1,0,1,0,4,0,0,37;39;43;27;43;50;38;33;22;28;30;27;40;49;61;2...,45;48;47;49;24,None,933,213,0,59,64,4,1,0.921875,1,1,0,0,61
4,2024,CAR,REG,17,341,547,3411,22,15,36,-223,3,1,4464,1431,171,-49.310815,-0.026900,0,410,1878,18,3,3,102,3.743683,1,341,518,3411,22,6,3,4464,1431,171,28.487255,0,0,650,72,566,65,299,8,32.0,223.0,64,9,96,67,0,1,0,37,7,7,8,8,0,118,947,73,24,177,42,1060,22,26,4,0,53,0.846154,0,7,8,6,1,0,0,0,1,2,1,0,43;38;43;35;26;48;27;24;40;48;53;36;30;32;29;3...,45;38;45;53,None,783,181,0,33,35,2,0,0.942857,1,1,0,0,36


The answer is to pull player stats for offensve positions and team stats for defensive positions. Will write a loop to pull all as pandas dataframes and csvs for full year and one with all info

In [44]:
df1s = []
df2s = []

for season in list(range(2014,2026)):
    df1 = nfl.load_team_stats(seasons=season, summary_level='reg').to_pandas()
    df2 = nfl.load_player_stats(seasons=season, summary_level='reg').to_pandas()
    df1s.append(df1)
    df2s.append(df2)
    # df1.to_csv(f'stats_team_nfl_{season}.csv')
    # df2.to_csv(f'stats_player_nfl_{season}.csv')

all_team_stats = pd.concat(df1s, ignore_index=True)
all_player_stats = pd.concat(df2s, ignore_index=True)
    
    

In [45]:
# all_team_stats.to_csv(path_or_buf='Raw/stats_team_nfl_raw_14to25.csv')
# all_player_stats.to_csv(path_or_buf='Raw/stats_player_nfl_raw_14to25.csv')

Need to get list of ids for merges in future

In [46]:
player_indexes = nfl.load_ff_playerids().to_pandas()

In [47]:
player_indexes.head()

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season
0,17030,3c76cab3-3df2-43dd-acaa-57e055bd32d0,24755.0,00-0040676,133244.0,12522.0,58203.0,4688380.0,None,None,3168422.0,WardCa00,None,16997.0,None,1730.0,41786.0,0.0,25323.0,None,Cam Ward,cam ward,QB,TEN,2002-05-25,23.4,2025.0,1.0,1.0,1.0,None,74.0,219.0,Miami (FL),2025
1,17031,270e09bc-8bf8-44b0-87ed-1fd014de4ab7,25968.0,00-0040668,131396.0,12524.0,58349.0,4432762.0,None,None,26710004.0,SandSh00,None,18479.0,None,1731.0,41930.0,0.0,26079.0,None,Shedeur Sanders,shedeur sanders,QB,CLE,2002-02-07,23.7,2025.0,5.0,6.0,144.0,None,74.0,212.0,Colorado,2025
2,17032,53848cba-bcad-4cd0-bb35-0a1f6ed111d1,23160.0,00-0040691,146409.0,12508.0,58227.0,4689114.0,None,None,26694019.0,DartJa00,None,18574.0,None,1732.0,41810.0,0.0,26082.0,None,Jaxson Dart,jaxson dart,QB,NYG,2003-05-13,22.5,2025.0,1.0,25.0,25.0,None,74.0,223.0,Ole Miss,2025
3,17033,b0eb5b67-4ef2-462d-bfe7-e73bcce1f6b0,22884.0,00-0040673,146590.0,12510.0,58297.0,4432734.0,None,None,26694943.0,MilrJa00,None,18477.0,None,1733.0,41882.0,0.0,25882.0,None,Jalen Milroe,jalen milroe,QB,SEA,2002-12-13,22.9,2025.0,3.0,28.0,92.0,None,74.0,216.0,Alabama,2025
4,17034,d89f0384-21a6-44e4-8aa6-324cb2103bf6,23680.0,00-0040234,127253.0,12500.0,58436.0,4889929.0,None,None,26746721.0,EwerQu00,None,18505.0,None,1734.0,42037.0,0.0,26048.0,None,Quinn Ewers,quinn ewers,QB,MIA,2003-05-13,22.5,2025.0,7.0,15.0,231.0,None,74.0,209.0,Texas,2025


In [48]:
player_indexes['yahoo_id'].nunique()

5482

In [49]:
player_indexes['yahoo_id'].dtype

dtype('O')

In [50]:
all_players['player_id'].nunique()

4263

In [51]:
all_players['player_id'].dtype

dtype('int64')

In [52]:
player_indexes = player_indexes.replace('None', np.nan)

In [53]:
all_players[all_players['player_id'].isin((player_indexes['yahoo_id'].fillna(-1).astype('int')))]

,bye_weeks.week,display_position,editorial_player_key,editorial_team_abbr,editorial_team_full_name,editorial_team_key,editorial_team_url,eligible_positions,headshot.size,headshot.url,image_url,is_keeper.status,is_keeper.cost,is_keeper.kept,is_undroppable,name.ascii_first,name.first,name.full,name.last,player_id,player_key,position_type,primary_position,url,season
0,10,K,nfl.p.3727,Ind,Indianapolis Colts,nfl.t.11,https://sports.yahoo.com/nfl/teams/indianapolis/,['K'],small,https://s.yimg.com/iu/api/res/1.2/OpHvpCHjl_PQ...,https://s.yimg.com/iu/api/res/1.2/OpHvpCHjl_PQ...,False,False,False,0,Adam,Adam,Adam Vinatieri,Vinatieri,3727,331.p.3727,K,K,https://sports.yahoo.com/nfl/players/3727,2014
1,9,TE,nfl.p.3950,Atl,Atlanta Falcons,nfl.t.1,https://sports.yahoo.com/nfl/teams/atlanta/,['TE'],small,https://s.yimg.com/iu/api/res/1.2/1_ZQRTBuF_fZ...,https://s.yimg.com/iu/api/res/1.2/1_ZQRTBuF_fZ...,False,False,False,0,Tony,Tony,Tony Gonzalez,Gonzalez,3950,331.p.3950,O,TE,https://sports.yahoo.com/nfl/players/3950,2014
2,4,QB,nfl.p.4256,Den,Denver Broncos,nfl.t.7,https://sports.yahoo.com/nfl/teams/denver/,['QB'],small,https://s.yimg.com/iu/api/res/1.2/UCWtIm9kEbbH...,https://s.yimg.com/iu/api/res/1.2/UCWtIm9kEbbH...,False,False,False,0,Peyton,Peyton,Peyton Manning,Manning,4256,331.p.4256,O,QB,https://sports.yahoo.com/nfl/players/4256,2014
3,4,K,nfl.p.4269,Ari,Arizona Cardinals,nfl.t.22,https://sports.yahoo.com/nfl/teams/arizona/,['K'],small,https://s.yimg.com/iu/api/res/1.2/z4ki.EZ6BaZp...,https://s.yimg.com/iu/api/res/1.2/z4ki.EZ6BaZp...,False,False,False,0,Phil,Phil,Phil Dawson,Dawson,4269,331.p.4269,K,K,https://sports.yahoo.com/nfl/players/4269,2014
4,10,QB,nfl.p.4416,Ind,Indianapolis Colts,nfl.t.11,https://sports.yahoo.com/nfl/teams/indianapolis/,['QB'],small,https://s.yimg.com/iu/api/res/1.2/YdWdJ6P7MDCj...,https://s.yimg.com/iu/api/res/1.2/YdWdJ6P7MDCj...,False,False,False,0,Matt,Matt,Matt Hasselbeck,Hasselbeck,4416,331.p.4416,O,QB,https://sports.yahoo.com/nfl/players/4416,2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13746,9,WR,nfl.p.41421,NYJ,New York Jets,nfl.t.20,https://sports.yahoo.com/nfl/teams/ny-jets/,['WR'],small,https://s.yimg.com/iu/api/res/1.2/FAXEqtpRun02...,https://s.yimg.com/iu/api/res/1.2/FAXEqtpRun02...,False,False,False,0,Isaiah,Isaiah,Isaiah Williams,Williams,41421,461.p.41421,O,WR,https://sports.yahoo.com/nfl/players/41421,2025
13749,8,WR,nfl.p.41453,LAR,Los Angeles Rams,nfl.t.14,https://sports.yahoo.com/nfl/teams/la-rams/,['WR'],small,https://s.yimg.com/iu/api/res/1.2/MDSo5Qtc99iQ...,https://s.yimg.com/iu/api/res/1.2/MDSo5Qtc99iQ...,False,False,False,0,Drake,Drake,Drake Stoops,Stoops,41453,461.p.41453,O,WR,https://sports.yahoo.com/nfl/players/41453,2025
13752,10,RB,nfl.p.41464,KC,Kansas City Chiefs,nfl.t.12,https://sports.yahoo.com/nfl/teams/kansas-city/,['RB'],small,https://s.yimg.com/iu/api/res/1.2/F_qrbQ3UN2fP...,https://s.yimg.com/iu/api/res/1.2/F_qrbQ3UN2fP...,False,False,False,0,Carson,Carson,Carson Steele,Steele,41464,461.p.41464,O,RB,https://sports.yahoo.com/nfl/players/41464,2025
13753,8,QB,nfl.p.41476,Ari,Arizona Cardinals,nfl.t.22,https://sports.yahoo.com/nfl/teams/arizona/,['QB'],small,https://s.yimg.com/iu/api/res/1.2/TcM85WhJ.fAO...,https://s.yimg.com/iu/api/res/1.2/TcM85WhJ.fAO...,False,False,False,0,Kedon,Kedon,Kedon Slovis,Slovis,41476,461.p.41476,O,QB,https://sports.yahoo.com/nfl/players/41476,2025


In [54]:
player_indexes[player_indexes['yahoo_id'].isna()].tail(100)

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season
12049,36,None,NaN,None,NaN,NaN,NaN,NaN,None,None,NaN,None,None,NaN,None,NaN,NaN,NaN,NaN,None,San Francisco 49ers,san francisco 49ers,DT,SFO,None,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None,2025
12050,42,None,NaN,None,NaN,NaN,NaN,NaN,None,None,NaN,None,None,NaN,None,NaN,NaN,NaN,NaN,None,Karim Abdul-Jabbar,karim abdul-jabbar,RB,IND,None,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None,2025
12051,48,None,NaN,None,NaN,NaN,NaN,NaN,None,None,NaN,None,None,NaN,None,NaN,NaN,NaN,NaN,None,Rabih Abdullah,rabih abdullah,RB,TBB,None,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None,2025
12052,54,None,NaN,None,NaN,NaN,NaN,NaN,None,None,NaN,None,None,NaN,None,NaN,NaN,NaN,NaN,None,Rahim Abdullah,rahim abdullah,LB,CLE,None,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None,2025
12053,60,None,NaN,None,NaN,NaN,NaN,NaN,None,None,NaN,None,None,NaN,None,NaN,NaN,NaN,NaN,None,Donnie Abraham,donnie abraham,DB,TBB,None,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None,2025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12144,972,None,NaN,None,NaN,NaN,NaN,NaN,None,None,NaN,None,None,NaN,None,NaN,NaN,NaN,NaN,None,Doug Brien,doug brien,PK,NOS,None,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None,2025
12145,978,None,NaN,None,NaN,NaN,NaN,NaN,None,None,NaN,None,None,NaN,None,NaN,NaN,NaN,NaN,None,Jeremy Brigham,jeremy brigham,TE,OAK,None,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None,2025
12146,984,None,NaN,None,NaN,NaN,NaN,NaN,None,None,NaN,None,None,NaN,None,NaN,NaN,NaN,NaN,None,Vincent Brisby,vincent brisby,WR,FA,None,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None,2025
12147,990,None,NaN,None,NaN,NaN,NaN,NaN,None,None,NaN,None,None,NaN,None,NaN,NaN,NaN,NaN,None,Bubby Brister,bubby brister,QB,KCC,None,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None,2025


The player index has more unique yahoo IDs than the yahoo list, which is good because hopefully all the ones in the yahoo database are covered. There are some 2025 rookies missing a yahoo id and many are players who retired prior to the 2014 season I am starting with

Solution -- will save the CSV for now and figure out the IDs fully in the clean/merge stage, or when it comes up as imperative to move forward with project

In [55]:
# player_indexes.to_csv('Raw/player_keys_raw.csv')

# Work with scraping ancillary data from pro football reference

In [56]:
pfr_url = 'https://www.pro-football-reference.com/teams/ram/coaches.htm'

html_test = pd.read_html(pfr_url)[1]

In [57]:
html_test.head()

Unnamed: 0_level_0 Unnamed: 1_level_0 Reg. Season                    \
                Year              Coach           G   W   L  T   W-L%   
0             2025.0         Sean McVay           7   5   2  0  0.714   
1             2024.0         Sean McVay          17  10   7  0  0.588   
2             2023.0         Sean McVay          17  10   7  0  0.588   
3             2022.0         Sean McVay          17   5  12  0  0.294   
4             2021.0         Sean McVay          17  12   5  0  0.706   

  Playoffs                         Coordinators                 
         G    W    L    Result          Offense        Defense  
0      NaN  NaN  NaN       NaN     Mike LaFleur    Chris Shula  
1      2.0  1.0  1.0  Lost Div     Mike LaFleur    Chris Shula  
2      1.0  0.0  1.0   Lost WC     Mike LaFleur  Raheem Morris  
3      NaN  NaN  NaN       NaN        Liam Coen  Raheem Morris  
4      4.0  4.0  0.0    Won SB  Kevin O'Connell  Raheem Morris

In [58]:
html_test.columns = html_test.columns.get_level_values(1)

In [59]:
html_test.head(10)

,Year,Coach,G,W,L,T,W-L%,G,W,L,Result,Offense,Defense
0,2025.0,Sean McVay,7,5,2,0,0.714,NaN,NaN,NaN,NaN,Mike LaFleur,Chris Shula
1,2024.0,Sean McVay,17,10,7,0,0.588,2.0,1.0,1.0,Lost Div,Mike LaFleur,Chris Shula
2,2023.0,Sean McVay,17,10,7,0,0.588,1.0,0.0,1.0,Lost WC,Mike LaFleur,Raheem Morris
3,2022.0,Sean McVay,17,5,12,0,0.294,NaN,NaN,NaN,NaN,Liam Coen,Raheem Morris
4,2021.0,Sean McVay,17,12,5,0,0.706,4.0,4.0,0.0,Won SB,Kevin O'Connell,Raheem Morris
5,2020.0,Sean McVay,16,10,6,0,0.625,2.0,1.0,1.0,Lost Div,Kevin O'Connell,Brandon Staley
6,2019.0,Sean McVay,16,9,7,0,0.563,NaN,NaN,NaN,NaN,NaN,Wade Phillips
7,2018.0,Sean McVay,16,13,3,0,0.813,3.0,2.0,1.0,Lost SB,NaN,Wade Phillips
8,2017.0,Sean McVay,16,11,5,0,0.688,1.0,0.0,1.0,Lost WC,Matt LaFleur,Wade Phillips
9,2016.0,John Fassel,3,0,3,0,0.000,NaN,NaN,NaN,NaN,Rob Boras,Gregg Williams


In [60]:
html_test.columns

Index(['Year', 'Coach', 'G', 'W', 'L', 'T', 'W-L%', 'G', 'W', 'L', 'Result',
       'Offense', 'Defense'],
      dtype='object')

In [61]:

html_test.head()

,Year,Coach,G,W,L,T,W-L%,G,W,L,Result,Offense,Defense
0,2025.0,Sean McVay,7,5,2,0,0.714,NaN,NaN,NaN,NaN,Mike LaFleur,Chris Shula
1,2024.0,Sean McVay,17,10,7,0,0.588,2.0,1.0,1.0,Lost Div,Mike LaFleur,Chris Shula
2,2023.0,Sean McVay,17,10,7,0,0.588,1.0,0.0,1.0,Lost WC,Mike LaFleur,Raheem Morris
3,2022.0,Sean McVay,17,5,12,0,0.294,NaN,NaN,NaN,NaN,Liam Coen,Raheem Morris
4,2021.0,Sean McVay,17,12,5,0,0.706,4.0,4.0,0.0,Won SB,Kevin O'Connell,Raheem Morris


In [62]:
html_test[html_test['Year'].isna()]

,Year,Coach,G,W,L,T,W-L%,G,W,L,Result,Offense,Defense
10,NaN,Jeff Fisher,13,4,9,0,0.308,0.0,0.0,0.0,NaN,Rob Boras,Gregg Williams
19,NaN,Scott Linehan,4,0,4,0,0.000,0.0,0.0,0.0,NaN,Al Saunders,"Jim Haslett, Rick Venturi"
23,NaN,Mike Martz,5,2,3,0,0.400,0.0,0.0,0.0,NaN,Steve Fairchild,Larry Marmie
67,NaN,Bob Waterfield,8,1,7,0,0.125,0.0,0.0,0.0,NaN,Hampton Pool,NaN
78,NaN,Joe Stydahar,1,0,1,0,0.000,0.0,0.0,0.0,Lost Div,NaN,NaN
92,NaN,Hugo Bezdek,3,0,3,0,0.000,0.0,0.0,0.0,NaN,NaN,NaN


In [63]:
pfr_teams = pd.read_csv('Raw/pfr_ids.csv')
pfr_teams.head()

,Team,Code,colon,apostrophe,full
0,Arizona Cardinals,crd,:,"""","""Arizona Cardinals"" : ""crd"""
1,Atlanta Falcons,atl,:,"""","""Atlanta Falcons"" : ""atl"""
2,Baltimore Ravens,rav,:,"""","""Baltimore Ravens"" : ""rav"""
3,Buffalo Bills,buf,:,"""","""Buffalo Bills"" : ""buf"""
4,Carolina Panthers,car,:,"""","""Carolina Panthers"" : ""car"""


In [64]:
pfr_dict = pfr_teams.set_index('Team')['Code'].to_dict()
display(pfr_dict)

{'Arizona Cardinals': 'crd',
 'Atlanta Falcons': 'atl',
 'Baltimore Ravens': 'rav',
 'Buffalo Bills': 'buf',
 'Carolina Panthers': 'car',
 'Chicago Bears': 'chi',
 'Cincinnati Bengals': 'cin',
 'Cleveland Browns': 'cle',
 'Dallas Cowboys': 'dal',
 'Denver Broncos': 'den',
 'Detroit Lions': 'det',
 'Green Bay Packers': 'gnb',
 'Houston Texans': 'htx',
 'Indianapolis Colts': 'clt',
 'Jacksonville Jaguars': 'jax',
 'Kansas City Chiefs': 'kan',
 'Las Vegas Raiders': 'rai',
 'Los Angeles Chargers': 'sdg',
 'Los Angeles Rams': 'ram',
 'Miami Dolphins': 'mia',
 'Minnesota Vikings': 'min',
 'New England Patriots': 'nwe',
 'New Orleans Saints': 'nor',
 'New York Giants': 'nyg',
 'New York Jets': 'nyj',
 'Philadelphia Eagles': 'phi',
 'Pittsburgh Steelers': 'pit',
 'San Francisco 49ers': 'sfo',
 'Seattle Seahawks': 'sea',
 'Tampa Bay Buccaneers': 'tam',
 'Tennessee Titans': 'oti',
 'Washington Commanders': 'was'}

In [68]:
dfs = []

for key, value in pfr_dict.items():
    print(f'{key}')
    pfr_url = f'https://www.pro-football-reference.com/teams/{value}/coaches.htm'
    team_coaches = pd.read_html(pfr_url)[1]
    team_coaches.columns = team_coaches.columns.get_level_values(1)
    team_coaches['Year'] = team_coaches['Year'].fillna(method='ffill').astype('int')
    team_coaches = team_coaches[['Year', 'G', 'Coach', 'Offense', 'Defense']]
    team_coaches['Team'] = key
    dfs.append(team_coaches)
    time.sleep(5)


Arizona Cardinals


C:\Users\nick_\AppData\Local\Temp\ipykernel_20184\2491203759.py:8: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  team_coaches['Year'] = team_coaches['Year'].fillna(method='ffill').astype('int')


Atlanta Falcons


C:\Users\nick_\AppData\Local\Temp\ipykernel_20184\2491203759.py:8: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  team_coaches['Year'] = team_coaches['Year'].fillna(method='ffill').astype('int')


Baltimore Ravens


...

C:\Users\nick_\AppData\Local\Temp\ipykernel_20184\2491203759.py:8: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  team_coaches['Year'] = team_coaches['Year'].fillna(method='ffill').astype('int')


Tennessee Titans


C:\Users\nick_\AppData\Local\Temp\ipykernel_20184\2491203759.py:8: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  team_coaches['Year'] = team_coaches['Year'].fillna(method='ffill').astype('int')


Washington Commanders


C:\Users\nick_\AppData\Local\Temp\ipykernel_20184\2491203759.py:8: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  team_coaches['Year'] = team_coaches['Year'].fillna(method='ffill').astype('int')


In [117]:
coaches = pd.concat(dfs, ignore_index=True)
coaches.head(50)
    

,Year,G,G,Coach,Offense,Defense,Team
0,2025,7,NaN,Jonathan Gannon,Drew Petzing,Nick Rallis,Arizona Cardinals
1,2024,17,NaN,Jonathan Gannon,Drew Petzing,Nick Rallis,Arizona Cardinals
2,2023,17,NaN,Jonathan Gannon,Drew Petzing,Nick Rallis,Arizona Cardinals
3,2022,17,NaN,Kliff Kingsbury,NaN,Vance Joseph,Arizona Cardinals
4,2021,17,1.0,Kliff Kingsbury,NaN,Vance Joseph,Arizona Cardinals
5,2020,16,NaN,Kliff Kingsbury,NaN,Vance Joseph,Arizona Cardinals
6,2019,16,NaN,Kliff Kingsbury,NaN,Vance Joseph,Arizona Cardinals
7,2018,16,NaN,Steve Wilks,"Byron Leftwich, Mike McCoy",Al Holcomb,Arizona Cardinals
8,2017,16,NaN,Bruce Arians,Harold Goodwin,James Bettcher,Arizona Cardinals
9,2016,16,NaN,Bruce Arians,Harold Goodwin,James Bettcher,Arizona Cardinals


In [118]:
coaches.to_csv(path_or_buf='Players/coaches_unedit.csv', index=False)


In [119]:

coaches1 =  pd.read_csv('Players/coaches_unedit.csv')

In [120]:
coaches1['G'].isna().sum()

np.int64(0)

In [121]:
coaches1 = coaches1.loc[:, ~coaches1.columns.duplicated()]
coaches1.head()

,Year,G,G.1,Coach,Offense,Defense,Team
0,2025,7,NaN,Jonathan Gannon,Drew Petzing,Nick Rallis,Arizona Cardinals
1,2024,17,NaN,Jonathan Gannon,Drew Petzing,Nick Rallis,Arizona Cardinals
2,2023,17,NaN,Jonathan Gannon,Drew Petzing,Nick Rallis,Arizona Cardinals
3,2022,17,NaN,Kliff Kingsbury,NaN,Vance Joseph,Arizona Cardinals
4,2021,17,1.0,Kliff Kingsbury,NaN,Vance Joseph,Arizona Cardinals


In [122]:
coaches1[coaches1['Team'] == 'Baltimore Ravens']

,Year,G,G.1,Coach,Offense,Defense,Team
184,2025,6,NaN,John Harbaugh,Todd Monken,Zach Orr,Baltimore Ravens
185,2024,17,2.0,John Harbaugh,Todd Monken,Zach Orr,Baltimore Ravens
186,2023,17,2.0,John Harbaugh,Todd Monken,Mike Macdonald,Baltimore Ravens
187,2022,17,1.0,John Harbaugh,Greg Roman,Mike Macdonald,Baltimore Ravens
188,2021,17,NaN,John Harbaugh,Greg Roman,Don Martindale,Baltimore Ravens
189,2020,16,2.0,John Harbaugh,Greg Roman,Don Martindale,Baltimore Ravens
190,2019,16,1.0,John Harbaugh,Greg Roman,Don Martindale,Baltimore Ravens
191,2018,16,1.0,John Harbaugh,Marty Mornhinweg,Don Martindale,Baltimore Ravens
192,2017,16,NaN,John Harbaugh,Marty Mornhinweg,Dean Pees,Baltimore Ravens
193,2016,16,NaN,John Harbaugh,"Marty Mornhinweg, Marc Trestman",Dean Pees,Baltimore Ravens


In [123]:
coaches1 = coaches1[coaches1['Year'] >= 2014]
coaches1['Year'].value_counts()

Year
2024    35
2023    35
2022    35
2020    35
2015    35
2016    35
2018    34
2021    34
2019    34
2025    33
2017    33
2014    33
Name: count, dtype: int64

In [124]:
coaches1 = coaches1.loc[coaches1.groupby(['Year', 'Team'])['G'].idxmax()]

In [125]:
coaches1.head(20)

,Year,G,G.1,Coach,Offense,Defense,Team
11,2014,16,1.0,Bruce Arians,Harold Goodwin,Todd Bowles,Arizona Cardinals
129,2014,16,0.0,Mike Smith,Dirk Koetter,Mike Nolan,Atlanta Falcons
195,2014,16,2.0,John Harbaugh,Gary Kubiak,Dean Pees,Baltimore Ravens
226,2014,16,0.0,Doug Marrone,Nathaniel Hackett,Jim Schwartz,Buffalo Bills
300,2014,16,2.0,Ron Rivera,Mike Shula,Sean McDermott,Carolina Panthers
332,2014,16,0.0,Marc Trestman,Aaron Kromer,Mel Tucker,Chicago Bears
443,2014,16,1.0,Marvin Lewis,Hue Jackson,Paul Guenther,Cincinnati Bengals
505,2014,16,0.0,Mike Pettine,Kyle Shanahan,Jim O'Neil,Cleveland Browns
586,2014,16,2.0,Jason Garrett,Bill Callahan,Rod Marinelli,Dallas Cowboys
654,2014,16,1.0,John Fox,Adam Gase,Jack Del Rio,Denver Broncos


In [126]:
coaches1.groupby('Year')['Team'].nunique()

Year
2014    32
2015    32
2016    32
2017    32
2018    32
2019    32
2020    32
2021    32
2022    32
2023    32
2024    32
2025    32
Name: Team, dtype: int64

In [127]:
coaches1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 384 entries, 11 to 2263
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Year     384 non-null    int64  
 1   G        384 non-null    int64  
 2   G.1      169 non-null    float64
 3   Coach    384 non-null    object 
 4   Offense  364 non-null    object 
 5   Defense  371 non-null    object 
 6   Team     384 non-null    object 
dtypes: float64(1), int64(2), object(4)
memory usage: 24.0+ KB


In [128]:
coaches1['Offense'] = coaches1['Offense'].fillna(coaches1['Coach'])

In [129]:
coaches1['Defense'] = coaches1['Defense'].fillna(coaches1['Coach'])

In [130]:
coaches1[coaches1['Team'] == 'New England Patriots']

,Year,G,G.1,Coach,Offense,Defense,Team
1511,2014,16,3.0,Bill Belichick,Josh McDaniels,Matt Patricia,New England Patriots
1510,2015,16,2.0,Bill Belichick,Josh McDaniels,Matt Patricia,New England Patriots
1509,2016,16,3.0,Bill Belichick,Josh McDaniels,Matt Patricia,New England Patriots
1508,2017,16,3.0,Bill Belichick,Josh McDaniels,Matt Patricia,New England Patriots
1507,2018,16,3.0,Bill Belichick,Josh McDaniels,Bill Belichick,New England Patriots
1506,2019,16,1.0,Bill Belichick,Josh McDaniels,Bill Belichick,New England Patriots
1505,2020,16,NaN,Bill Belichick,Josh McDaniels,Bill Belichick,New England Patriots
1504,2021,17,1.0,Bill Belichick,Josh McDaniels,Bill Belichick,New England Patriots
1503,2022,17,NaN,Bill Belichick,Bill Belichick,Bill Belichick,New England Patriots
1502,2023,17,NaN,Bill Belichick,Bill O'Brien,Bill Belichick,New England Patriots


In [131]:
team_abbrev = {
    'Arizona Cardinals': 'ARI',
    'Atlanta Falcons': 'ATL',
    'Baltimore Ravens': 'BAL',
    'Buffalo Bills': 'BUF',
    'Carolina Panthers': 'CAR',
    'Chicago Bears': 'CHI',
    'Cincinnati Bengals': 'CIN',
    'Cleveland Browns': 'CLE',
    'Dallas Cowboys': 'DAL',
    'Denver Broncos': 'DEN',
    'Detroit Lions': 'DET',
    'Green Bay Packers': 'GB',
    'Houston Texans': 'HOU',
    'Indianapolis Colts': 'IND',
    'Jacksonville Jaguars': 'JAX',
    'Kansas City Chiefs': 'KC',
    'Las Vegas Raiders': 'LV',
    'Los Angeles Chargers': 'LAC',
    'Los Angeles Rams': 'LAR',
    'Miami Dolphins': 'MIA',
    'Minnesota Vikings': 'MIN',
    'New England Patriots': 'NE',
    'New Orleans Saints': 'NO',
    'New York Giants': 'NYG',
    'New York Jets': 'NYJ',
    'Philadelphia Eagles': 'PHI',
    'Pittsburgh Steelers': 'PIT',
    'San Francisco 49ers': 'SF',
    'Seattle Seahawks': 'SEA',
    'Tampa Bay Buccaneers': 'TB',
    'Tennessee Titans': 'TEN',
    'Washington Commanders': 'WAS'
}

In [132]:
coaches1['team_code'] = coaches1['Team'].map(team_abbrev)

In [133]:
coaches1.head()

,Year,G,G.1,Coach,Offense,Defense,Team,team_code
11,2014,16,1.0,Bruce Arians,Harold Goodwin,Todd Bowles,Arizona Cardinals,ARI
129,2014,16,0.0,Mike Smith,Dirk Koetter,Mike Nolan,Atlanta Falcons,ATL
195,2014,16,2.0,John Harbaugh,Gary Kubiak,Dean Pees,Baltimore Ravens,BAL
226,2014,16,0.0,Doug Marrone,Nathaniel Hackett,Jim Schwartz,Buffalo Bills,BUF
300,2014,16,2.0,Ron Rivera,Mike Shula,Sean McDermott,Carolina Panthers,CAR


In [134]:
coaches1[coaches1['Offense'].str.contains(',')]

,Year,G,G.1,Coach,Offense,Defense,Team,team_code
724,2015,16,NaN,Jim Caldwell,"Jim Bob Cooter, Joe Lombardi",Teryl Austin,Detroit Lions,DET
960,2015,16,NaN,Chuck Pagano,"Rob Chudzinski, Pep Hamilton",Greg Manusky,Indianapolis Colts,IND
1287,2015,16,NaN,Jeff Fisher,"Rob Boras, Frank Cignetti",Gregg Williams,Los Angeles Rams,LAR
1380,2015,12,NaN,Dan Campbell,"Bill Lazor, Zac Taylor","Lou Anarumo, Kevin Coyle",Miami Dolphins,MIA
193,2016,16,NaN,John Harbaugh,"Marty Mornhinweg, Marc Trestman",Dean Pees,Baltimore Ravens,BAL
224,2016,15,0.0,Rex Ryan,"Anthony Lynn, Greg Roman",Dennis Thurman,Buffalo Bills,BUF
1039,2016,14,0.0,Gus Bradley,"Nathaniel Hackett, Greg Olson",Todd Wash,Jacksonville Jaguars,JAX
1442,2016,16,NaN,Mike Zimmer,"Pat Shurmur, Norv Turner",George Edwards,Minnesota Vikings,MIN
440,2017,16,NaN,Marvin Lewis,"Bill Lazor, Ken Zampese",Paul Guenther,Cincinnati Bengals,CIN
651,2017,16,NaN,Vance Joseph,"Mike McCoy, Bill Musgrave",Joe Woods,Denver Broncos,DEN


In [135]:
coaches1['Offense'] = coaches1['Offense'].str.split(',').str[-1].str.strip()
coaches1['Defense'] = coaches1['Defense'].str.split(',').str[-1].str.strip()

In [136]:
coaches1.loc[724]

Year                  2015
G                       16
G.1                    NaN
Coach         Jim Caldwell
Offense       Joe Lombardi
Defense       Teryl Austin
Team         Detroit Lions
team_code              DET
Name: 724, dtype: object

In [137]:
coaches1 = coaches1.drop(labels='G.1', axis=1)
coaches1.head()

,Year,G,Coach,Offense,Defense,Team,team_code
11,2014,16,Bruce Arians,Harold Goodwin,Todd Bowles,Arizona Cardinals,ARI
129,2014,16,Mike Smith,Dirk Koetter,Mike Nolan,Atlanta Falcons,ATL
195,2014,16,John Harbaugh,Gary Kubiak,Dean Pees,Baltimore Ravens,BAL
226,2014,16,Doug Marrone,Nathaniel Hackett,Jim Schwartz,Buffalo Bills,BUF
300,2014,16,Ron Rivera,Mike Shula,Sean McDermott,Carolina Panthers,CAR


In [138]:

# coaches.to_csv(path_or_buf='Raw/coaches.csv')

In [139]:

pfr_def_url = 'https://www.pro-football-reference.com/years/2024/opp.htm#all_team_stats'

defender = pd.read_html(pfr_def_url)[0]
defender.columns

MultiIndex([( 'Unnamed: 0_level_0',    'Rk'),
            ( 'Unnamed: 1_level_0',    'Tm'),
            ( 'Unnamed: 2_level_0',     'G'),
            ( 'Unnamed: 3_level_0',    'PA'),
            ( 'Unnamed: 4_level_0',   'Yds'),
            (       'Tot Yds & TO',   'Ply'),
            (       'Tot Yds & TO',   'Y/P'),
            (       'Tot Yds & TO',    'TO'),
            ( 'Unnamed: 8_level_0',    'FL'),
            ( 'Unnamed: 9_level_0',  '1stD'),
            (            'Passing',   'Cmp'),
            (            'Passing',   'Att'),
            (            'Passing',   'Yds'),
            (            'Passing',    'TD'),
            (            'Passing',   'Int'),
            (            'Passing',  'NY/A'),
            (            'Passing',  '1stD'),
            (            'Rushing',   'Att'),
            (            'Rushing',   'Yds'),
            (            'Rushing',    'TD'),
            (            'Rushing',   'Y/A'),
            (            'Rushing'

In [140]:
defender.columns = ['.'.join(col).strip() for col in defender.columns.values]

In [141]:
defender = defender.rename(columns={'Unnamed: 1_level_0.Tm':'Team','Unnamed: 3_level_0.PA':'Tot Yds & TO.PA',
                             'Unnamed: 4_level_0.Yds': 'Tot Yds & TO.Yds', 'Unnamed: 8_level_0.FL': 'Tot Yds & TO.FR',
                             'Unnamed: 9_level_0.1stD' : 'Tot Yds & TO.1stD',  'Unnamed: 25_level_0.Sc%': 'Tot Yds & TO.Sc%',
                             'Unnamed: 26_level_0.TO%' : 'Tot Yds & TO.TO%', 'Unnamed: 27_level_0.EXP' : 'Tot Yds & TO.EXP'}) \
                             .drop(labels=['Unnamed: 0_level_0.Rk', 'Unnamed: 2_level_0.G'], axis=1)


In [142]:

defender.head()

,Team,Tot Yds & TO.PA,Tot Yds & TO.Yds,Tot Yds & TO.Ply,Tot Yds & TO.Y/P,Tot Yds & TO.TO,Tot Yds & TO.FR,Tot Yds & TO.1stD,Passing.Cmp,Passing.Att,Passing.Yds,Passing.TD,Passing.Int,Passing.NY/A,Passing.1stD,Rushing.Att,Rushing.Yds,Rushing.TD,Rushing.Y/A,Rushing.1stD,Penalties.Pen,Penalties.Yds,Penalties.1stPy,Tot Yds & TO.Sc%,Tot Yds & TO.TO%,Tot Yds & TO.EXP
0,Los Angeles Chargers,301.0,5514.0,1037.0,5.3,21.0,6.0,311.0,367.0,565.0,3517.0,24.0,15.0,5.8,170.0,426.0,1997.0,7.0,4.7,108.0,101.0,869.0,33.0,32.6,11.6,-12.31
1,Philadelphia Eagles,303.0,4732.0,999.0,4.7,26.0,13.0,293.0,337.0,542.0,2961.0,22.0,13.0,5.1,160.0,416.0,1771.0,9.0,4.3,108.0,85.0,694.0,25.0,31.6,14.1,76.38
2,Denver Broncos,311.0,5391.0,1093.0,4.9,25.0,10.0,327.0,393.0,606.0,3752.0,22.0,15.0,5.6,203.0,424.0,1639.0,10.0,3.9,82.0,101.0,723.0,42.0,31.6,12.4,34.83
3,Kansas City Chiefs,326.0,5451.0,1026.0,5.3,20.0,7.0,328.0,375.0,569.0,3720.0,24.0,13.0,6.1,191.0,418.0,1731.0,13.0,4.1,112.0,107.0,849.0,25.0,36.5,11.4,-74.11
4,Minnesota Vikings,332.0,5702.0,1078.0,5.3,33.0,9.0,325.0,415.0,637.0,4114.0,24.0,24.0,6.0,218.0,392.0,1588.0,12.0,4.1,83.0,141.0,1185.0,24.0,31.6,16.6,35.95


In [154]:
dfs = []

for season in list(range(2014,2026)):
    pfr_def_url = f'https://www.pro-football-reference.com/years/{season}/opp.htm#all_team_stats'
    print(f'{season} processing')
    df = pd.read_html(pfr_def_url)[0]
    df.columns = ['.'.join(col).strip() for col in df.columns.values]
    df = df.rename(columns={'Unnamed: 1_level_0.Tm':'Team','Unnamed: 3_level_0.PA':'Tot Yds & TO.PA',
                             'Unnamed: 4_level_0.Yds': 'Tot Yds & TO.Yds', 'Unnamed: 8_level_0.FL': 'Tot Yds & TO.FR',
                             'Unnamed: 9_level_0.1stD' : 'Tot Yds & TO.1stD',  'Unnamed: 25_level_0.Sc%': 'Tot Yds & TO.Sc%',
                             'Unnamed: 26_level_0.TO%' : 'Tot Yds & TO.TO%', 'Unnamed: 27_level_0.EXP' : 'Tot Yds & TO.EXP'}) \
                             .drop(labels=['Unnamed: 0_level_0.Rk', 'Unnamed: 2_level_0.G'], axis=1)
    df['season'] = season
    dfs.append(df)
    print(f'{season} complete!')
    time.sleep(5)


2014 processing
2014 complete!
2015 processing
2015 complete!
2016 processing
2016 complete!
2017 processing
2017 complete!
2018 processing
2018 complete!
2019 processing
2019 complete!
2020 processing
2020 complete!
2021 processing
2021 complete!
2022 processing
2022 complete!
2023 processing
2023 complete!
2024 processing
2024 complete!
2025 processing
2025 complete!


In [155]:
def_stats = pd.concat(dfs, ignore_index=True)
def_stats.head()

,Team,Tot Yds & TO.PA,Tot Yds & TO.Yds,Tot Yds & TO.Ply,Tot Yds & TO.Y/P,Tot Yds & TO.TO,Tot Yds & TO.FR,Tot Yds & TO.1stD,Passing.Cmp,Passing.Att,Passing.Yds,Passing.TD,Passing.Int,Passing.NY/A,Passing.1stD,Rushing.Att,Rushing.Yds,Rushing.TD,Rushing.Y/A,Rushing.1stD,Penalties.Pen,Penalties.Yds,Penalties.1stPy,Tot Yds & TO.Sc%,Tot Yds & TO.TO%,Tot Yds & TO.EXP,season
0,Seattle Seahawks,254.0,4274.0,924.0,4.6,24.0,11.0,277.0,313.0,507.0,2970.0,17.0,13.0,5.5,175.0,380.0,1304.0,8.0,3.4,70.0,70.0,613.0,32.0,26.9,12.3,40.92,2014
1,Kansas City Chiefs,281.0,5288.0,1024.0,5.2,14.0,8.0,310.0,318.0,545.0,3252.0,22.0,6.0,5.5,184.0,433.0,2036.0,4.0,4.7,108.0,107.0,851.0,18.0,32.6,6.7,-16.38,2014
2,Detroit Lions,282.0,4815.0,984.0,4.9,27.0,7.0,310.0,383.0,592.0,3706.0,23.0,20.0,5.8,209.0,350.0,1109.0,8.0,3.2,59.0,99.0,849.0,42.0,26.9,14.3,53.86,2014
3,Buffalo Bills,289.0,4995.0,1027.0,4.9,30.0,11.0,302.0,337.0,559.0,3292.0,16.0,19.0,5.4,171.0,414.0,1703.0,10.0,4.1,89.0,104.0,849.0,42.0,28.3,14.1,43.28,2014
4,Arizona Cardinals,299.0,5891.0,1010.0,5.8,25.0,7.0,300.0,365.0,579.0,4152.0,22.0,18.0,6.8,195.0,396.0,1739.0,9.0,4.4,77.0,130.0,1192.0,28.0,29.8,12.8,-9.75,2014


In [156]:
def_stats.to_csv('Raw/stats_def_nfl_raw_14to25.csv', index=False)

In [157]:
def_stats = pd.read_csv('Raw/stats_def_nfl_raw_14to25.csv')
def_stats.head()

,Team,Tot Yds & TO.PA,Tot Yds & TO.Yds,Tot Yds & TO.Ply,Tot Yds & TO.Y/P,Tot Yds & TO.TO,Tot Yds & TO.FR,Tot Yds & TO.1stD,Passing.Cmp,Passing.Att,Passing.Yds,Passing.TD,Passing.Int,Passing.NY/A,Passing.1stD,Rushing.Att,Rushing.Yds,Rushing.TD,Rushing.Y/A,Rushing.1stD,Penalties.Pen,Penalties.Yds,Penalties.1stPy,Tot Yds & TO.Sc%,Tot Yds & TO.TO%,Tot Yds & TO.EXP,season
0,Seattle Seahawks,254.0,4274.0,924.0,4.6,24.0,11.0,277.0,313.0,507.0,2970.0,17.0,13.0,5.5,175.0,380.0,1304.0,8.0,3.4,70.0,70.0,613.0,32.0,26.9,12.3,40.92,2014
1,Kansas City Chiefs,281.0,5288.0,1024.0,5.2,14.0,8.0,310.0,318.0,545.0,3252.0,22.0,6.0,5.5,184.0,433.0,2036.0,4.0,4.7,108.0,107.0,851.0,18.0,32.6,6.7,-16.38,2014
2,Detroit Lions,282.0,4815.0,984.0,4.9,27.0,7.0,310.0,383.0,592.0,3706.0,23.0,20.0,5.8,209.0,350.0,1109.0,8.0,3.2,59.0,99.0,849.0,42.0,26.9,14.3,53.86,2014
3,Buffalo Bills,289.0,4995.0,1027.0,4.9,30.0,11.0,302.0,337.0,559.0,3292.0,16.0,19.0,5.4,171.0,414.0,1703.0,10.0,4.1,89.0,104.0,849.0,42.0,28.3,14.1,43.28,2014
4,Arizona Cardinals,299.0,5891.0,1010.0,5.8,25.0,7.0,300.0,365.0,579.0,4152.0,22.0,18.0,6.8,195.0,396.0,1739.0,9.0,4.4,77.0,130.0,1192.0,28.0,29.8,12.8,-9.75,2014


In [158]:
all_team_stats.head()

,season,team,season_type,games,completions,attempts,passing_yards,passing_tds,passing_interceptions,sacks_suffered,sack_yards_lost,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,passing_epa,passing_cpoe,passing_2pt_conversions,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_epa,rushing_2pt_conversions,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,special_teams_tds,def_tackles_solo,def_tackles_with_assist,def_tackle_assists,def_tackles_for_loss,def_tackles_for_loss_yards,def_fumbles_forced,def_sacks,def_sack_yards,def_qb_hits,def_interceptions,def_interception_yards,def_pass_defended,def_tds,def_fumbles,def_safeties,misc_yards,fumble_recovery_own,fumble_recovery_yards_own,fumble_recovery_opp,fumble_recovery_yards_opp,fumble_recovery_tds,penalties,penalty_yards,timeouts,punt_returns,punt_return_yards,kickoff_returns,kickoff_return_yards,fg_made,fg_att,fg_missed,fg_blocked,fg_long,fg_pct,fg_made_0_19,fg_made_20_29,fg_made_30_39,fg_made_40_49,fg_made_50_59,fg_made_60_,fg_missed_0_19,fg_missed_20_29,fg_missed_30_39,fg_missed_40_49,fg_missed_50_59,fg_missed_60_,fg_made_list,fg_missed_list,fg_blocked_list,fg_made_distance,fg_missed_distance,fg_blocked_distance,pat_made,pat_att,pat_missed,pat_blocked,pat_pct,gwfg_made,gwfg_att,gwfg_missed,gwfg_blocked,gwfg_distance_list,full_team
0,2014,ARI,REG,16,320,568,3990,21,12,28,-182,3,1,6028,1604,192,18.740342,-4.518646,1,397,1308,6,4,2,67,-65.700445,1,320,561,3990,21,5,2,6028,1604,192,58.771839,1,1,710,51,161,82,361,9,35.0,244.0,83,18,321,62,3,2,0,62,9,9,7,7,1,91,707,51,26,277,22,417,29,33,4,0,51,0.878788,0,5,12,10,2,0,0,0,2,1,1,0,22;44;49;37;32;33;51;32;35;33;48;33;49;37;41;2...,53;49;36;34,None,1089,172,0,27,27,0,0,1.00000,0,0,0,0,None,Arizona Cardinals
1,2014,ATL,REG,16,418,632,4758,28,15,31,-205,4,2,4896,2099,233,73.115751,1.520407,1,372,1498,11,4,2,79,-45.943996,0,418,623,4758,28,7,4,4896,2099,233,124.410778,1,1,645,41,337,55,211,13,22.0,140.0,63,16,113,66,1,1,0,75,8,9,12,12,1,97,840,64,18,239,50,1174,29,32,2,1,54,0.906250,0,10,4,8,7,0,0,0,0,0,2,0,40;51;52;46;20;20;52;54;37;43;44;33;34;42;26;4...,59;57,53,1075,116,53,40,40,0,0,1.00000,1,1,0,0,52,Atlanta Falcons
2,2014,BAL,REG,16,344,554,3986,27,12,19,-167,1,0,4623,1751,194,72.347553,-0.729385,0,448,2019,16,5,2,116,-25.763741,1,344,541,3986,27,4,3,4623,1751,194,100.047974,0,2,645,24,372,74,407,16,49.0,362.0,118,11,135,75,1,3,1,45,7,9,11,11,0,111,870,68,32,292,38,1077,29,34,4,1,55,0.852941,0,10,11,4,4,0,0,0,0,0,4,0,38;30;23;22;20;38;21;32;30;52;27;46;49;38;38;4...,55;57;54;54,64,1006,220,64,42,42,0,0,1.00000,1,1,0,0,32,Baltimore Ravens
3,2014,BUF,REG,16,363,579,3856,23,13,39,-242,4,3,4173,2077,186,-6.710150,-1.833058,1,402,1482,7,5,2,66,-58.216538,1,363,572,3856,23,7,3,4173,2077,186,51.756559,1,3,618,146,304,88,354,14,54.0,308.0,99,19,322,75,0,5,1,14,9,10,11,11,1,124,1031,54,37,354,33,770,34,38,3,1,58,0.894737,0,9,12,7,6,0,0,0,1,1,1,0,50;33;27;27;27;31;32;38;45;31;45;25;58;31;53;3...,31;50;47,53,1277,128,53,31,32,1,0,0.96875,2,2,0,0,27;58,Buffalo Bills
4,2014,CAR,REG,16,327,545,3828,23,12,42,-317,6,4,5321,1427,199,14.271482,-1.083109,1,473,2036,10,9,4,126,-29.101412,0,327,540,3828,23,3,1,5321,1427,199,86.374184,1,1,640,15,315,64,278,22,40.0,268.0,80,14,350,73,2,6,0,0,14,18,12,12,0,83,756,55,30,239,37,805,29,35,5,1,53,0.828571,0,7,12,9,1,0,0,0,1,3,1,0,28;33;29;53;38;40;40;37;44;39;44;36;33;31;26;4...,48;38;46;50;49,63,1047,231,63,34,34,0,0,1.00000,0,1,0,1,63,Carolina Panthers


In [159]:

reversed_abbrev = {v: k for k, v in team_abbrev.items()}

reversed_abbrev

{'ARI': 'Arizona Cardinals',
 'ATL': 'Atlanta Falcons',
 'BAL': 'Baltimore Ravens',
 'BUF': 'Buffalo Bills',
 'CAR': 'Carolina Panthers',
 'CHI': 'Chicago Bears',
 'CIN': 'Cincinnati Bengals',
 'CLE': 'Cleveland Browns',
 'DAL': 'Dallas Cowboys',
 'DEN': 'Denver Broncos',
 'DET': 'Detroit Lions',
 'GB': 'Green Bay Packers',
 'HOU': 'Houston Texans',
 'IND': 'Indianapolis Colts',
 'JAX': 'Jacksonville Jaguars',
 'KC': 'Kansas City Chiefs',
 'LV': 'Las Vegas Raiders',
 'LAC': 'Los Angeles Chargers',
 'LAR': 'Los Angeles Rams',
 'MIA': 'Miami Dolphins',
 'MIN': 'Minnesota Vikings',
 'NE': 'New England Patriots',
 'NO': 'New Orleans Saints',
 'NYG': 'New York Giants',
 'NYJ': 'New York Jets',
 'PHI': 'Philadelphia Eagles',
 'PIT': 'Pittsburgh Steelers',
 'SF': 'San Francisco 49ers',
 'SEA': 'Seattle Seahawks',
 'TB': 'Tampa Bay Buccaneers',
 'TEN': 'Tennessee Titans',
 'WAS': 'Washington Commanders'}

In [160]:
all_team_stats['full_team'] = all_team_stats['team'].map(reversed_abbrev)

In [161]:
all_team_stats.head()

,season,team,season_type,games,completions,attempts,passing_yards,passing_tds,passing_interceptions,sacks_suffered,sack_yards_lost,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,passing_epa,passing_cpoe,passing_2pt_conversions,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_epa,rushing_2pt_conversions,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,special_teams_tds,def_tackles_solo,def_tackles_with_assist,def_tackle_assists,def_tackles_for_loss,def_tackles_for_loss_yards,def_fumbles_forced,def_sacks,def_sack_yards,def_qb_hits,def_interceptions,def_interception_yards,def_pass_defended,def_tds,def_fumbles,def_safeties,misc_yards,fumble_recovery_own,fumble_recovery_yards_own,fumble_recovery_opp,fumble_recovery_yards_opp,fumble_recovery_tds,penalties,penalty_yards,timeouts,punt_returns,punt_return_yards,kickoff_returns,kickoff_return_yards,fg_made,fg_att,fg_missed,fg_blocked,fg_long,fg_pct,fg_made_0_19,fg_made_20_29,fg_made_30_39,fg_made_40_49,fg_made_50_59,fg_made_60_,fg_missed_0_19,fg_missed_20_29,fg_missed_30_39,fg_missed_40_49,fg_missed_50_59,fg_missed_60_,fg_made_list,fg_missed_list,fg_blocked_list,fg_made_distance,fg_missed_distance,fg_blocked_distance,pat_made,pat_att,pat_missed,pat_blocked,pat_pct,gwfg_made,gwfg_att,gwfg_missed,gwfg_blocked,gwfg_distance_list,full_team
0,2014,ARI,REG,16,320,568,3990,21,12,28,-182,3,1,6028,1604,192,18.740342,-4.518646,1,397,1308,6,4,2,67,-65.700445,1,320,561,3990,21,5,2,6028,1604,192,58.771839,1,1,710,51,161,82,361,9,35.0,244.0,83,18,321,62,3,2,0,62,9,9,7,7,1,91,707,51,26,277,22,417,29,33,4,0,51,0.878788,0,5,12,10,2,0,0,0,2,1,1,0,22;44;49;37;32;33;51;32;35;33;48;33;49;37;41;2...,53;49;36;34,None,1089,172,0,27,27,0,0,1.00000,0,0,0,0,None,Arizona Cardinals
1,2014,ATL,REG,16,418,632,4758,28,15,31,-205,4,2,4896,2099,233,73.115751,1.520407,1,372,1498,11,4,2,79,-45.943996,0,418,623,4758,28,7,4,4896,2099,233,124.410778,1,1,645,41,337,55,211,13,22.0,140.0,63,16,113,66,1,1,0,75,8,9,12,12,1,97,840,64,18,239,50,1174,29,32,2,1,54,0.906250,0,10,4,8,7,0,0,0,0,0,2,0,40;51;52;46;20;20;52;54;37;43;44;33;34;42;26;4...,59;57,53,1075,116,53,40,40,0,0,1.00000,1,1,0,0,52,Atlanta Falcons
2,2014,BAL,REG,16,344,554,3986,27,12,19,-167,1,0,4623,1751,194,72.347553,-0.729385,0,448,2019,16,5,2,116,-25.763741,1,344,541,3986,27,4,3,4623,1751,194,100.047974,0,2,645,24,372,74,407,16,49.0,362.0,118,11,135,75,1,3,1,45,7,9,11,11,0,111,870,68,32,292,38,1077,29,34,4,1,55,0.852941,0,10,11,4,4,0,0,0,0,0,4,0,38;30;23;22;20;38;21;32;30;52;27;46;49;38;38;4...,55;57;54;54,64,1006,220,64,42,42,0,0,1.00000,1,1,0,0,32,Baltimore Ravens
3,2014,BUF,REG,16,363,579,3856,23,13,39,-242,4,3,4173,2077,186,-6.710150,-1.833058,1,402,1482,7,5,2,66,-58.216538,1,363,572,3856,23,7,3,4173,2077,186,51.756559,1,3,618,146,304,88,354,14,54.0,308.0,99,19,322,75,0,5,1,14,9,10,11,11,1,124,1031,54,37,354,33,770,34,38,3,1,58,0.894737,0,9,12,7,6,0,0,0,1,1,1,0,50;33;27;27;27;31;32;38;45;31;45;25;58;31;53;3...,31;50;47,53,1277,128,53,31,32,1,0,0.96875,2,2,0,0,27;58,Buffalo Bills
4,2014,CAR,REG,16,327,545,3828,23,12,42,-317,6,4,5321,1427,199,14.271482,-1.083109,1,473,2036,10,9,4,126,-29.101412,0,327,540,3828,23,3,1,5321,1427,199,86.374184,1,1,640,15,315,64,278,22,40.0,268.0,80,14,350,73,2,6,0,0,14,18,12,12,0,83,756,55,30,239,37,805,29,35,5,1,53,0.828571,0,7,12,9,1,0,0,0,1,3,1,0,28;33;29;53;38;40;40;37;44;39;44;36;33;31;26;4...,48;38;46;50;49,63,1047,231,63,34,34,0,0,1.00000,0,1,0,1,63,Carolina Panthers


In [183]:
depth_2025 = nfl.load_depth_charts(seasons=2025).to_pandas()
depth_2025.head()


,dt,team,player_name,espn_id,gsis_id,pos_grp_id,pos_grp,pos_id,pos_name,pos_abb,pos_slot,pos_rank
0,2025-10-24T07:15:22Z,ARI,Josh Sweat,3693166,00-0034381,16,Base 4-3 D,11,Left Defensive End,LDE,1,1
1,2025-10-24T07:15:22Z,ARI,Calais Campbell,11284,00-0026190,16,Base 4-3 D,24,Left Defensive Tackle,LDT,2,1
2,2025-10-24T07:15:22Z,ARI,Dalvin Tomlinson,2979860,00-0033546,16,Base 4-3 D,25,Right Defensive Tackle,RDT,3,1
3,2025-10-24T07:15:22Z,ARI,Darius Robinson,4569480,00-0039854,16,Base 4-3 D,13,Right Defensive End,RDE,4,1
4,2025-10-24T07:15:22Z,ARI,Akeem Davis-Gaither,3917142,00-0036298,16,Base 4-3 D,26,Weakside Linebacker,WLB,5,1


In [168]:
depth_2025.to_csv('Players/depth_2025.csv', index=False)

In [174]:
dfs = []

for season in list(range(2014,2025)):
    print(f'Processing {season}...')
    df = nfl.load_depth_charts(seasons=season).to_pandas()
    dfs.append(df)
    df.to_csv(f'Players/depth_{season}.csv', index=False)
    print(f'{season} complete!')
    time.sleep(3)


Processing 2014...
2014 complete!
Processing 2015...
2015 complete!
Processing 2016...
2016 complete!
Processing 2017...
2017 complete!
Processing 2018...
2018 complete!
Processing 2019...
2019 complete!
Processing 2020...
2020 complete!
Processing 2021...
2021 complete!
Processing 2022...
2022 complete!
Processing 2023...
2023 complete!
Processing 2024...
2024 complete!


In [180]:


depth_charts = pd.concat(dfs, ignore_index=True)


In [182]:
depth_charts['season'].value_counts()

season
2022    37780
2021    37487
2023    37327
2024    37312
2015    37058
2017    36620
2016    36612
2018    36560
2019    36308
2020    36168
2014    32542
Name: count, dtype: int64

In [184]:
depth_charts.to_csv(path_or_buf='Players/depth_14to24.csv', index=False)

In [185]:
import requests
from io import StringIO


In [186]:
pfr_def_url

'https://www.pro-football-reference.com/years/2025/opp.htm#all_team_stats'

In [ ]:
# dfs = []

# for season in list(range(2014,2026)):
#     print(f'Processing {season}...')
#     # Fetch the page
#     response = requests.get(f'https://www.pro-football-reference.com/years/{season}/opp.htm#all_team_stats')
#     html_content = response.text

#     # Remove HTML comments that hide tables
#     html_content = html_content.replace('<!--', '').replace('-->', '')

#     # Now read all tables
#     all_tables = pd.read_html(StringIO(html_content))

#     print(f"Total tables found: {len(all_tables)}")

#     # Access your table (you may need to try different indices)
#     if season >= 2018:
#         conversions = all_tables[8]  # or try other indices
#     else:
#         conversions = all_tables[7]
#     conversions.columns = conversions.columns.get_level_values(1)
#     conversions['season'] = season
#     dfs.append(conversions)
#     conversions.to_csv(path_or_buf=f'Stats/stats_conversions_{season}.csv', index=False)
#     print(f'{season} complete!')

Processing 2014...
Total tables found: 9
2014 complete!
Processing 2015...
Total tables found: 9
2015 complete!
Processing 2016...
Total tables found: 9
2016 complete!
Processing 2017...
Total tables found: 9
2017 complete!
Processing 2018...
Total tables found: 10
2018 complete!
Processing 2019...
Total tables found: 10
2019 complete!
Processing 2020...
Total tables found: 10
2020 complete!
Processing 2021...
Total tables found: 10
2021 complete!
Processing 2022...
Total tables found: 10
2022 complete!
Processing 2023...
Total tables found: 10
2023 complete!
Processing 2024...
Total tables found: 10
2024 complete!
Processing 2025...
Total tables found: 10
2025 complete!


In [211]:
all_conversions = pd.concat(dfs, ignore_index=True)
all_conversions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 396 entries, 0 to 395
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Rk      384 non-null    float64
 1   Tm      396 non-null    object 
 2   G       384 non-null    float64
 3   3DAtt   384 non-null    float64
 4   3DConv  384 non-null    float64
 5   3D%     384 non-null    object 
 6   4DAtt   384 non-null    float64
 7   4DConv  384 non-null    float64
 8   4D%     384 non-null    object 
 9   RZAtt   384 non-null    float64
 10  RZTD    384 non-null    float64
 11  RZPct   384 non-null    object 
 12  season  396 non-null    int64  
dtypes: float64(8), int64(1), object(4)
memory usage: 40.3+ KB


In [213]:
all_conversions.to_csv(f'Raw/conversions_raw_14to25.csv', index=False)
all_conversions = pd.read_csv('Raw/conversions_raw_14to25.csv')

In [214]:
all_conversions.head()

,Rk,Tm,G,3DAtt,3DConv,3D%,4DAtt,4DConv,4D%,RZAtt,RZTD,RZPct,season
0,1.0,Indianapolis Colts,16.0,201.0,67.0,33.3%,17.0,4.0,23.5%,50.0,33.0,66.0%,2014
1,2.0,Pittsburgh Steelers,16.0,189.0,71.0,37.6%,14.0,10.0,71.4%,50.0,27.0,54.0%,2014
2,3.0,New York Giants,16.0,200.0,72.0,36.0%,10.0,4.0,40.0%,51.0,25.0,49.0%,2014
3,4.0,Buffalo Bills,16.0,220.0,73.0,33.2%,11.0,5.0,45.5%,44.0,22.0,50.0%,2014
4,5.0,Houston Texans,16.0,218.0,74.0,33.9%,17.0,6.0,35.3%,46.0,22.0,47.8%,2014
